In [21]:
# Create an enhanced v2 notebook with constraints, multipliers, and constructor modeling.
# File: F1_Fantasy_Vegas_Model_v2.ipynb

import nbformat as nbf
from pathlib import Path

nb = nbf.v4.new_notebook()
cells = []

cells.append(nbf.v4.new_markdown_cell("""
# F1 Fantasy — GP Las Vegas (2025) — **VEPI & Team Optimizer v2**
Recursos adicionais:
- **Locks** (sempre incluir) e **Exclusions** (nunca incluir) para **pilotos** e **construtores**  
- **Multiplicadores** por piloto (ex.: capitão 2×, mega 3×)  
- **Modelagem de Construtor**: pontos = **soma dos 2 pilotos da equipe** + **pit points** (faixas e bônus)  
- **1 ou 2 construtores** (parametrizável) dentro do **orçamento total**

> Observação: este otimizador não usa odds; ele combina histórico técnico (VEPI) + seus inputs Fantasy.
"""))

cells.append(nbf.v4.new_code_cell("""
# =====================
# Parâmetros principais
# =====================

# Diretório dos race traces locais
RACE_TRACES_DIR = "f1_race_traces_2021"  # ajuste se necessário

# Excel gerado anteriormente com lista de pilotos (opcional)
FANTASY_INPUT_XLSX = "F1_Fantasy_GP_Sao_Paulo_2025_3Camadas.xlsx"  # ajuste se desejar

# Saída
OUT_EXCEL = "F1_Fantasy_Vegas_Projections_v2.xlsx"

# Corridas similares a Vegas (urbana/noturna)
URBAN_NIGHT_TRACKS = {"Las Vegas", "Jeddah", "Miami", "Singapore", "Baku"}

# Sprint weekend de Vegas?
SPRINT_WEEKEND = False
SPRINT_UPLIFT = 1.25

# Orçamento total (pilotos + construtor(es))
TOTAL_BUDGET = 100.0

# Quantidade de construtores (1 padrão; pode usar 2 se seu jogo permitir)
NUM_CONSTRUCTORS = 1  # mude para 2 se quiser

# Locks/Exclusions — escreva exatamente como no campo "Driver" / nome do construtor
LOCK_DRIVERS = []                 # ex.: ["Oscar Piastri"]
EXCLUDE_DRIVERS = []              # ex.: ["Jack Doohan"]
LOCK_CONSTRUCTORS = []            # ex.: ["McLaren"]
EXCLUDE_CONSTRUCTORS = []         # ex.: []

# Multiplicadores por piloto (nome -> fator). Ex.: {"Oscar Piastri": 2.0}
DRIVER_MULTIPLIERS = {
    # "Oscar Piastri": 2.0,
}
"""))

cells.append(nbf.v4.new_code_cell("""
import os, glob, random
import pandas as pd
import numpy as np
from itertools import combinations

SEED = 42
random.seed(SEED); np.random.seed(SEED)

def safe_read(path):
    try:
        if path.lower().endswith('.csv'):
            return pd.read_csv(path)
        if path.lower().endswith('.parquet'):
            return pd.read_parquet(path)
        if path.lower().endswith('.json'):
            return pd.read_json(path, lines=False)
    except Exception as e:
        print(f"[warn] erro ao ler {path}: {e}")
    return None

def coalesce(series, default=np.nan):
    return series if series is not None else default
"""))

cells.append(nbf.v4.new_code_cell("""
# =============
# 1) RACE TRACES
# =============
files = []
if os.path.isdir(RACE_TRACES_DIR):
    for ext in ('*.csv', '*.parquet', '*.json'):
        files.extend(glob.glob(os.path.join(RACE_TRACES_DIR, ext)))

dfs = []
for f in files:
    df = safe_read(f)
    if df is None or df.empty:
        continue

    # Inferência de corrida pelo nome do arquivo
    fname = os.path.basename(f)
    race_guess = None
    for name in ["Las Vegas","Jeddah","Miami","Singapore","Baku","Monaco","Abu Dhabi","Austin","Mexico City","Sao Paulo","Interlagos"]:
        if name.replace(" ", "_").lower() in fname.lower() or name.lower() in fname.lower():
            race_guess = name
            break

    # Mapear colunas possíveis
    driver = None
    for cand in ["driver","Driver","driver_name","abbr"]:
        if cand in df.columns:
            driver = df[cand].astype(str)
            break

    team = None
    for cand in ["team","constructor","Team","Constructor"]:
        if cand in df.columns:
            team = df[cand].astype(str)
            break

    start_pos = None
    for cand in ["grid","start_position","grid_position","startPos"]:
        if cand in df.columns:
            start_pos = pd.to_numeric(df[cand], errors='coerce')
            break

    finish_pos = None
    for cand in ["position","finish_position","classified_position","Pos"]:
        if cand in df.columns:
            finish_pos = pd.to_numeric(df[cand], errors='coerce')
            break

    fastest_lap_flag = None
    for cand in ["fastest_lap","is_fastest_lap","fastestLap"]:
        if cand in df.columns:
            fastest_lap_flag = (df[cand].astype(str).str.lower().isin(["1","true","yes"])).astype(int)
            break

    dnf_flag = None
    for cand in ["dnf","did_not_finish","retired","not_classified"]:
        if cand in df.columns:
            dnf_flag = (df[cand].astype(str).str.lower().isin(["1","true","yes"])).astype(int)
            break

    pit_time = None
    for cand in ["pit_time","pitstop_time","pitstopDuration","PitTime"]:
        if cand in df.columns:
            pit_time = pd.to_numeric(df[cand], errors='coerce')
            break

    if driver is None or (start_pos is None and finish_pos is None):
        continue

    out = pd.DataFrame({
        "driver": coalesce(driver),
        "team": coalesce(team),
        "race": race_guess,
        "start_position": coalesce(start_pos),
        "finish_position": coalesce(finish_pos),
        "fastest_lap": coalesce(fastest_lap_flag, 0),
        "dnf": coalesce(dnf_flag, 0),
        "pit_time": coalesce(pit_time, np.nan),
    })
    dfs.append(out)

if dfs:
    df_all = pd.concat(dfs, ignore_index=True)
else:
    df_all = pd.DataFrame(columns=["driver","team","race","start_position","finish_position","fastest_lap","dnf","pit_time"])

for col in ["start_position","finish_position","fastest_lap","dnf","pit_time"]:
    if col in df_all.columns:
        df_all[col] = pd.to_numeric(df_all[col], errors='coerce')

# Agregação por piloto/corrida
if not df_all.empty:
    df_all["positions_gained"] = np.where(
        df_all["start_position"].notna() & df_all["finish_position"].notna(),
        (df_all["start_position"] - df_all["finish_position"]), np.nan
    )
    pit_summary = df_all.groupby(["driver","race"]).agg(
        pitstop_best=("pit_time","min"),
        pitstop_mean=("pit_time","mean")
    ).reset_index() if "pit_time" in df_all.columns else pd.DataFrame()

    base = df_all.groupby(["driver","race"]).agg(
        finish_position=("finish_position","min"),
        start_position=("start_position","min"),
        positions_gained=("positions_gained","mean"),
        fastest_lap=("fastest_lap","max"),
        dnf=("dnf","max"),
        laps_count=("finish_position","count")
    ).reset_index()

    if not pit_summary.empty:
        df_summary = base.merge(pit_summary, on=["driver","race"], how="left")
    else:
        df_summary = base.copy()
else:
    df_summary = pd.DataFrame(columns=["driver","race","finish_position","start_position","positions_gained","fastest_lap","dnf","laps_count","pitstop_best","pitstop_mean"])

df_summary.head(5)"""))

cells.append(nbf.v4.new_code_cell("""
# =============
# 2) VEPI
# =============
if not df_summary.empty:
    df_urban = df_summary[df_summary["race"].isin(URBAN_NIGHT_TRACKS)].copy()
    if df_urban.empty:
        print("[aviso] Sem corridas urbanas/noturnas identificadas, usando fallback com todas as corridas.")
        df_urban = df_summary.copy()

    vepi = df_urban.groupby("driver").agg(
        mean_positions_gained=("positions_gained","mean"),
        mean_finish_position=("finish_position","mean"),
        fastest_lap_count=("fastest_lap","sum"),
        dnf_rate=("dnf","mean"),
        pit_best_mean=("pitstop_best","mean"),
    )
    vepi = vepi.fillna({"mean_positions_gained":0, "dnf_rate":0})
    vepi["score_finish"] = 25 - vepi["mean_finish_position"].fillna(20)
    vepi["score_gain"] = vepi["mean_positions_gained"] * 0.8
    vepi["score_fastlap"] = (vepi["fastest_lap_count"] > 0).astype(int) * 5
    vepi["score_dnf"] = - vepi["dnf_rate"] * 20
    vepi["VEPI"] = vepi[["score_finish","score_gain","score_fastlap","score_dnf"]].sum(axis=1)
else:
    vepi = pd.DataFrame(columns=["driver","VEPI"])

vepi.sort_values("VEPI", ascending=False).head(10)"""))

cells.append(nbf.v4.new_code_cell("""
# =============
# 3) Inputs Fantasy (fallback se não houver Excel)
# =============
fallback_data = [
    ("Lando Norris", 30.4, 30.70),
    ("Oscar Piastri", 26.0, 29.25),
    ("Max Verstappen", 29.2, 27.80),
    ("George Russell", 23.4, 23.85),
    ("Lewis Hamilton", 22.8, 17.30),
    ("Charles Leclerc", 23.3, 17.30),
    ("Kimi Antonelli", 16.3, 10.55),
    ("Alexander Albon", 13.2, 9.05),
    ("Oliver Bearman", 8.3, 8.85),
    ("Yuki Tsunoda", 10.8, 8.00),
    ("Lance Stroll", 8.5, 7.25),
    ("Nico Hülkenberg", 6.8, 6.25),
    ("Esteban Ocon", 6.5, 6.20),
    ("Isack Hadjar", 5.9, 4.15),
    ("Gabriel Bortoleto", 7.3, 3.85),
    ("Carlos Sainz", 6.3, 3.45),
    ("Liam Lawson", 16.8, 2.50),
    ("Pierre Gasly", 4.7, 1.65),
    ("Franco Colapinto", 4.7, 1.29),
    ("Fernando Alonso", 5.5, 1.25),
    ("Jack Doohan", 4.5, 0.00),
]
columns = ["Driver","Cost ($M)","Avg Pts/GP"]

if os.path.exists(FANTASY_INPUT_XLSX):
    try:
        df_inputs = pd.read_excel(FANTASY_INPUT_XLSX, sheet_name="1-Inputs (Drivers)")
    except Exception as e:
        print(f"[warn] Erro lendo {FANTASY_INPUT_XLSX}: {e}")
        df_inputs = pd.DataFrame(fallback_data, columns=columns)
else:
    df_inputs = pd.DataFrame(fallback_data, columns=columns)

if SPRINT_WEEKEND:
    df_inputs["Proj Pts (Event)"] = (df_inputs["Avg Pts/GP"] * SPRINT_UPLIFT).round(2)
else:
    df_inputs["Proj Pts (Event)"] = df_inputs["Avg Pts/GP"].round(2)

# Merge VEPI (ajuste percentual por desvio-padrão)
if not vepi.empty:
    vepi_norm = vepi[["VEPI"]].copy()
    vepi_norm["VEPI_z"] = (vepi_norm["VEPI"] - vepi_norm["VEPI"].mean()) / (vepi_norm["VEPI"].std(ddof=0) + 1e-6)
    vepi_norm["VEPI_adj"] = 1 + (vepi_norm["VEPI_z"] * 0.10)
    merged = df_inputs.merge(vepi_norm[["VEPI_adj"]], left_on="Driver", right_index=True, how="left")
    merged["VEPI_adj"].fillna(1.0, inplace=True)
else:
    merged = df_inputs.copy()
    merged["VEPI_adj"] = 1.0

merged["Projected Vegas Pts (no mult)"] = (merged["Proj Pts (Event)"] * merged["VEPI_adj"]).round(2)
merged.head(8)"""))

cells.append(nbf.v4.new_code_cell("""
# =============
# 4) Mapear times e configurar construtores
# =============
# Mapeamento leve 2025 (ajuste se necessário)
TEAM_BY_DRIVER = {
    "Lando Norris": "McLaren",
    "Oscar Piastri": "McLaren",
    "Max Verstappen": "Red Bull Racing",
    "Liam Lawson": "Red Bull Racing",
    "George Russell": "Mercedes",
    "Kimi Antonelli": "Mercedes",
    "Lewis Hamilton": "Ferrari",
    "Charles Leclerc": "Ferrari",
    "Alexander Albon": "Williams",
    "Franco Colapinto": "Williams",
    "Nico Hülkenberg": "Kick Sauber",
    "Oliver Bearman": "Haas",  # ajustar se necessário
    "Esteban Ocon": "Alpine",
    "Pierre Gasly": "Alpine",
    "Isack Hadjar": "RB",
    "Yuki Tsunoda": "RB",
    "Lance Stroll": "Aston Martin",
    "Fernando Alonso": "Aston Martin",
    "Gabriel Bortoleto": "???",
    "Carlos Sainz": "???",
    "Jack Doohan": "Alpine",
}

merged["Team"] = merged["Driver"].map(TEAM_BY_DRIVER).fillna("Unknown")

# Parâmetros de pit dos construtores (faixas e bônus). Ajuste conforme seu modelo.
CONSTRUCTOR_COSTS = {
    "McLaren": 35.6,
    "Red Bull Racing": 30.2,
    "Ferrari": 33.0,
    "Mercedes": 32.0,
    "Williams": 18.0,
    "Aston Martin": 20.0,
    "Alpine": 15.0,
    "RB": 14.0,
    "Kick Sauber": 12.0,
    "Haas": 11.0,
    "Unknown": 10.0,
}

# Estimativas de pit para Vegas (pode substituir por valores reais quando tiver)
# Base + bônus (fastest 5, world record 15)
CONSTRUCTOR_PIT_POINTS = {
    "McLaren": {"base": 10, "fastest": 5, "world_record": 0},
    "Red Bull Racing": {"base": 5, "fastest": 0, "world_record": 0},
    "Ferrari": {"base": 5, "fastest": 0, "world_record": 0},
    "Mercedes": {"base": 2, "fastest": 0, "world_record": 0},
    "Williams": {"base": 0, "fastest": 0, "world_record": 0},
    "Aston Martin": {"base": 2, "fastest": 0, "world_record": 0},
    "Alpine": {"base": 0, "fastest": 0, "world_record": 0},
    "RB": {"base": 2, "fastest": 0, "world_record": 0},
    "Kick Sauber": {"base": 0, "fastest": 0, "world_record": 0},
    "Haas": {"base": 0, "fastest": 0, "world_record": 0},
    "Unknown": {"base": 0, "fastest": 0, "world_record": 0},
}

# Pontos estimados do construtor = soma(pts dos dois pilotos da equipe) + pit points
constructor_scores = []
for team, cost in CONSTRUCTOR_COSTS.items():
    team_drivers = merged[merged["Team"] == team]
    # Soma dos 2 melhores (ou de todos da equipe se 2 exatamente)
    top2 = team_drivers.nlargest(2, "Projected Vegas Pts (no mult)")
    drivers_sum = top2["Projected Vegas Pts (no mult)"].sum()
    pit = CONSTRUCTOR_PIT_POINTS.get(team, {"base":0,"fastest":0,"world_record":0})
    pit_points = pit["base"] + pit["fastest"] + pit["world_record"]
    constructor_scores.append((team, cost, drivers_sum, pit_points, drivers_sum + pit_points))

df_constructors = pd.DataFrame(constructor_scores, columns=["Constructor","Cost ($M)","Sum Drivers Pts","Pit Pts","Constructor Pts (Est.)"])

# Aplicar locks/exclusions para construtores
if LOCK_CONSTRUCTORS:
    df_constructors = df_constructors[df_constructors["Constructor"].isin(LOCK_CONSTRUCTORS + [c for c in df_constructors["Constructor"] if c not in EXCLUDE_CONSTRUCTORS])]
if EXCLUDE_CONSTRUCTORS:
    df_constructors = df_constructors[~df_constructors["Constructor"].isin(EXCLUDE_CONSTRUCTORS)]

df_constructors.sort_values("Constructor Pts (Est.)", ascending=False).head(5)"""))

cells.append(nbf.v4.new_code_cell("""
# =============
# 5) Otimizador com 5 pilotos + N construtores
# =============
cand = merged.dropna(subset=["Projected Vegas Pts (no mult)","Cost ($M)"]).copy()
cand = cand[cand["Cost ($M)"] > 0]

# Aplicar locks/exclusions de pilotos
if LOCK_DRIVERS:
    cand = cand[cand["Driver"].isin(LOCK_DRIVERS + [d for d in cand["Driver"] if d not in EXCLUDE_DRIVERS])]
if EXCLUDE_DRIVERS:
    cand = cand[~cand["Driver"].isin(EXCLUDE_DRIVERS)]

# Multiplicadores
def apply_mult(driver, pts):
    mult = DRIVER_MULTIPLIERS.get(driver, 1.0)
    return pts * mult

best = {"pts": -1, "drivers": None, "constructors": None, "cost": None}

driver_rows = list(cand.itertuples(index=False))
constructors_rows = list(df_constructors.itertuples(index=False))

def constructors_combos(rows, k):
    if k == 1:
        for r in rows:
            yield (r,)
    else:
        from itertools import combinations
        for combo in combinations(rows, k):
            yield combo

for drv_combo in combinations(driver_rows, 5):
    drivers_cost = sum(getattr(d, "_2") for d in drv_combo)  # Cost ($M)
    # early prune by budget (min constructor cost)
    min_const_cost = min([getattr(c, "_2") for c in constructors_rows]) * NUM_CONSTRUCTORS if constructors_rows else 0
    if drivers_cost + min_const_cost > TOTAL_BUDGET:
        continue

    drivers_pts = sum(apply_mult(d[0], getattr(d, "_6")) for d in drv_combo)  # Projected Vegas Pts (no mult) * mult

    for cons_combo in constructors_combos(constructors_rows, NUM_CONSTRUCTORS):
        cons_cost = sum(getattr(c, "_2") for c in cons_combo)
        total_cost = drivers_cost + cons_cost
        if total_cost > TOTAL_BUDGET:
            continue
        cons_pts = sum(getattr(c, "_5") for c in cons_combo)  # Constructor Pts (Est.)
        total_pts = drivers_pts + cons_pts
        if total_pts > best["pts"]:
            best = {"pts": total_pts, "drivers": drv_combo, "constructors": cons_combo, "cost": total_cost}

best_summary = None
if best["drivers"]:
    best_drivers_df = pd.DataFrame([{
        "Driver": d[0],
        "Team": d[8],
        "Cost ($M)": d[1],
        "Projected Vegas Pts (no mult)": d[6],
        "Multiplier": DRIVER_MULTIPLIERS.get(d[0], 1.0),
        "Projected Vegas Pts (with mult)": apply_mult(d[0], d[6]),
        "Value (pts/$M)": round(apply_mult(d[0], d[6]) / d[1], 3)
    } for d in best["drivers"]]).sort_values("Projected Vegas Pts (with mult)", ascending=False).reset_index(drop=True)

    best_constructors_df = pd.DataFrame([{
        "Constructor": c[0],
        "Cost ($M)": c[1],
        "Sum Drivers Pts": c[2],
        "Pit Pts": c[3],
        "Constructor Pts (Est.)": c[4]
    } for c in best["constructors"]]).sort_values("Constructor Pts (Est.)", ascending=False).reset_index(drop=True)

    best_summary = {
        "Total Cost ($M)": round(best["cost"], 2),
        "Total Projected Pts": round(best["pts"], 2),
        "Drivers Cost": round(best_drivers_df["Cost ($M)"].sum(), 2),
        "Constructors Cost": round(best_constructors_df["Cost ($M)"].sum(), 2),
        "Drivers Pts (with mult)": round(best_drivers_df["Projected Vegas Pts (with mult)"].sum(), 2),
        "Constructors Pts": round(best_constructors_df["Constructor Pts (Est.)"].sum(), 2),
    }

best_summary, best_drivers_df.head(10) if best_summary else None, best_constructors_df if best_summary else None"""))

cells.append(nbf.v4.new_code_cell("""
# =============
# 6) Exportar Excel v2
# =============
with pd.ExcelWriter(OUT_EXCEL, engine="xlsxwriter") as writer:
    merged.sort_values("Projected Vegas Pts (no mult)", ascending=False).to_excel(writer, sheet_name="Drivers Projections", index=False)
    if 'vepi' in globals() and not vepi.empty:
        vepi.reset_index().to_excel(writer, sheet_name="VEPI", index=False)
    df_constructors.sort_values("Constructor Pts (Est.)", ascending=False).to_excel(writer, sheet_name="Constructors", index=False)
    if best_summary:
        best_drivers_df.to_excel(writer, sheet_name="Best Drivers", index=False)
        best_constructors_df.to_excel(writer, sheet_name="Best Constructors", index=False)
        pd.DataFrame([best_summary]).to_excel(writer, sheet_name="Best Summary", index=False)

OUT_EXCEL"""))

nb['cells'] = cells

from pathlib import Path
import nbformat as nbf

ipynb_path = Path("F1_Fantasy_Vegas_Model_v2.ipynb")  # salva no diretório atual
with open(ipynb_path, "w", encoding="utf-8") as f:
    nbf.write(nb, f)

print(f"Notebook salvo em: {ipynb_path.resolve()}")


Notebook salvo em: C:\F1\setup\f1_race_traces_2021\F1_Fantasy_Vegas_Model_v2.ipynb


In [19]:
# Create an enhanced v2 notebook with constraints, multipliers, and constructor modeling.
# File: F1_Fantasy_Vegas_Model_v2.ipynb

import nbformat as nbf
from pathlib import Path

nb = nbf.v4.new_notebook()
cells = []

cells.append(nbf.v4.new_markdown_cell("""
# F1 Fantasy — GP Las Vegas (2025) — **VEPI & Team Optimizer v2**
Recursos adicionais:
- **Locks** (sempre incluir) e **Exclusions** (nunca incluir) para **pilotos** e **construtores**  
- **Multiplicadores** por piloto (ex.: capitão 2×, mega 3×)  
- **Modelagem de Construtor**: pontos = **soma dos 2 pilotos da equipe** + **pit points** (faixas e bônus)  
- **1 ou 2 construtores** (parametrizável) dentro do **orçamento total**

> Observação: este otimizador não usa odds; ele combina histórico técnico (VEPI) + seus inputs Fantasy.
"""))

cells.append(nbf.v4.new_code_cell("""
# =====================
# Parâmetros principais
# =====================

# Diretório dos race traces locais
RACE_TRACES_DIR = "f1_race_traces_2021"  # ajuste se necessário

# Excel gerado anteriormente com lista de pilotos (opcional)
FANTASY_INPUT_XLSX = "F1_Fantasy_GP_Sao_Paulo_2025_3Camadas.xlsx"  # ajuste se desejar

# Saída
OUT_EXCEL = "F1_Fantasy_Vegas_Projections_v2.xlsx"

# Corridas similares a Vegas (urbana/noturna)
URBAN_NIGHT_TRACKS = {"Las Vegas", "Jeddah", "Miami", "Singapore", "Baku"}

# Sprint weekend de Vegas?
SPRINT_WEEKEND = False
SPRINT_UPLIFT = 1.25

# Orçamento total (pilotos + construtor(es))
TOTAL_BUDGET = 100.0

# Quantidade de construtores (1 padrão; pode usar 2 se seu jogo permitir)
NUM_CONSTRUCTORS = 1  # mude para 2 se quiser

# Locks/Exclusions — escreva exatamente como no campo "Driver" / nome do construtor
LOCK_DRIVERS = []                 # ex.: ["Oscar Piastri"]
EXCLUDE_DRIVERS = []              # ex.: ["Jack Doohan"]
LOCK_CONSTRUCTORS = []            # ex.: ["McLaren"]
EXCLUDE_CONSTRUCTORS = []         # ex.: []

# Multiplicadores por piloto (nome -> fator). Ex.: {"Oscar Piastri": 2.0}
DRIVER_MULTIPLIERS = {
    # "Oscar Piastri": 2.0,
}
"""))

cells.append(nbf.v4.new_code_cell("""
import os, glob, random
import pandas as pd
import numpy as np
from itertools import combinations

SEED = 42
random.seed(SEED); np.random.seed(SEED)

def safe_read(path):
    try:
        if path.lower().endswith('.csv'):
            return pd.read_csv(path)
        if path.lower().endswith('.parquet'):
            return pd.read_parquet(path)
        if path.lower().endswith('.json'):
            return pd.read_json(path, lines=False)
    except Exception as e:
        print(f"[warn] erro ao ler {path}: {e}")
    return None

def coalesce(series, default=np.nan):
    return series if series is not None else default
"""))

cells.append(nbf.v4.new_code_cell("""
# =============
# 1) RACE TRACES
# =============
files = []
if os.path.isdir(RACE_TRACES_DIR):
    for ext in ('*.csv', '*.parquet', '*.json'):
        files.extend(glob.glob(os.path.join(RACE_TRACES_DIR, ext)))

dfs = []
for f in files:
    df = safe_read(f)
    if df is None or df.empty:
        continue

    # Inferência de corrida pelo nome do arquivo
    fname = os.path.basename(f)
    race_guess = None
    for name in ["Las Vegas","Jeddah","Miami","Singapore","Baku","Monaco","Abu Dhabi","Austin","Mexico City","Sao Paulo","Interlagos"]:
        if name.replace(" ", "_").lower() in fname.lower() or name.lower() in fname.lower():
            race_guess = name
            break

    # Mapear colunas possíveis
    driver = None
    for cand in ["driver","Driver","driver_name","abbr"]:
        if cand in df.columns:
            driver = df[cand].astype(str)
            break

    team = None
    for cand in ["team","constructor","Team","Constructor"]:
        if cand in df.columns:
            team = df[cand].astype(str)
            break

    start_pos = None
    for cand in ["grid","start_position","grid_position","startPos"]:
        if cand in df.columns:
            start_pos = pd.to_numeric(df[cand], errors='coerce')
            break

    finish_pos = None
    for cand in ["position","finish_position","classified_position","Pos"]:
        if cand in df.columns:
            finish_pos = pd.to_numeric(df[cand], errors='coerce')
            break

    fastest_lap_flag = None
    for cand in ["fastest_lap","is_fastest_lap","fastestLap"]:
        if cand in df.columns:
            fastest_lap_flag = (df[cand].astype(str).str.lower().isin(["1","true","yes"])).astype(int)
            break

    dnf_flag = None
    for cand in ["dnf","did_not_finish","retired","not_classified"]:
        if cand in df.columns:
            dnf_flag = (df[cand].astype(str).str.lower().isin(["1","true","yes"])).astype(int)
            break

    pit_time = None
    for cand in ["pit_time","pitstop_time","pitstopDuration","PitTime"]:
        if cand in df.columns:
            pit_time = pd.to_numeric(df[cand], errors='coerce')
            break

    if driver is None or (start_pos is None and finish_pos is None):
        continue

    out = pd.DataFrame({
        "driver": coalesce(driver),
        "team": coalesce(team),
        "race": race_guess,
        "start_position": coalesce(start_pos),
        "finish_position": coalesce(finish_pos),
        "fastest_lap": coalesce(fastest_lap_flag, 0),
        "dnf": coalesce(dnf_flag, 0),
        "pit_time": coalesce(pit_time, np.nan),
    })
    dfs.append(out)

if dfs:
    df_all = pd.concat(dfs, ignore_index=True)
else:
    df_all = pd.DataFrame(columns=["driver","team","race","start_position","finish_position","fastest_lap","dnf","pit_time"])

for col in ["start_position","finish_position","fastest_lap","dnf","pit_time"]:
    if col in df_all.columns:
        df_all[col] = pd.to_numeric(df_all[col], errors='coerce')

# Agregação por piloto/corrida
if not df_all.empty:
    df_all["positions_gained"] = np.where(
        df_all["start_position"].notna() & df_all["finish_position"].notna(),
        (df_all["start_position"] - df_all["finish_position"]), np.nan
    )
    pit_summary = df_all.groupby(["driver","race"]).agg(
        pitstop_best=("pit_time","min"),
        pitstop_mean=("pit_time","mean")
    ).reset_index() if "pit_time" in df_all.columns else pd.DataFrame()

    base = df_all.groupby(["driver","race"]).agg(
        finish_position=("finish_position","min"),
        start_position=("start_position","min"),
        positions_gained=("positions_gained","mean"),
        fastest_lap=("fastest_lap","max"),
        dnf=("dnf","max"),
        laps_count=("finish_position","count")
    ).reset_index()

    if not pit_summary.empty:
        df_summary = base.merge(pit_summary, on=["driver","race"], how="left")
    else:
        df_summary = base.copy()
else:
    df_summary = pd.DataFrame(columns=["driver","race","finish_position","start_position","positions_gained","fastest_lap","dnf","laps_count","pitstop_best","pitstop_mean"])

df_summary.head(5)"""))

cells.append(nbf.v4.new_code_cell("""
# =============
# 2) VEPI
# =============
if not df_summary.empty:
    df_urban = df_summary[df_summary["race"].isin(URBAN_NIGHT_TRACKS)].copy()
    if df_urban.empty:
        print("[aviso] Sem corridas urbanas/noturnas identificadas, usando fallback com todas as corridas.")
        df_urban = df_summary.copy()

    vepi = df_urban.groupby("driver").agg(
        mean_positions_gained=("positions_gained","mean"),
        mean_finish_position=("finish_position","mean"),
        fastest_lap_count=("fastest_lap","sum"),
        dnf_rate=("dnf","mean"),
        pit_best_mean=("pitstop_best","mean"),
    )
    vepi = vepi.fillna({"mean_positions_gained":0, "dnf_rate":0})
    vepi["score_finish"] = 25 - vepi["mean_finish_position"].fillna(20)
    vepi["score_gain"] = vepi["mean_positions_gained"] * 0.8
    vepi["score_fastlap"] = (vepi["fastest_lap_count"] > 0).astype(int) * 5
    vepi["score_dnf"] = - vepi["dnf_rate"] * 20
    vepi["VEPI"] = vepi[["score_finish","score_gain","score_fastlap","score_dnf"]].sum(axis=1)
else:
    vepi = pd.DataFrame(columns=["driver","VEPI"])

vepi.sort_values("VEPI", ascending=False).head(10)"""))

cells.append(nbf.v4.new_code_cell("""
# =============
# 3) Inputs Fantasy (fallback se não houver Excel)
# =============
fallback_data = [
    ("Lando Norris", 30.4, 30.70),
    ("Oscar Piastri", 26.0, 29.25),
    ("Max Verstappen", 29.2, 27.80),
    ("George Russell", 23.4, 23.85),
    ("Lewis Hamilton", 22.8, 17.30),
    ("Charles Leclerc", 23.3, 17.30),
    ("Kimi Antonelli", 16.3, 10.55),
    ("Alexander Albon", 13.2, 9.05),
    ("Oliver Bearman", 8.3, 8.85),
    ("Yuki Tsunoda", 10.8, 8.00),
    ("Lance Stroll", 8.5, 7.25),
    ("Nico Hülkenberg", 6.8, 6.25),
    ("Esteban Ocon", 6.5, 6.20),
    ("Isack Hadjar", 5.9, 4.15),
    ("Gabriel Bortoleto", 7.3, 3.85),
    ("Carlos Sainz", 6.3, 3.45),
    ("Liam Lawson", 16.8, 2.50),
    ("Pierre Gasly", 4.7, 1.65),
    ("Franco Colapinto", 4.7, 1.29),
    ("Fernando Alonso", 5.5, 1.25),
    ("Jack Doohan", 4.5, 0.00),
]
columns = ["Driver","Cost ($M)","Avg Pts/GP"]

if os.path.exists(FANTASY_INPUT_XLSX):
    try:
        df_inputs = pd.read_excel(FANTASY_INPUT_XLSX, sheet_name="1-Inputs (Drivers)")
    except Exception as e:
        print(f"[warn] Erro lendo {FANTASY_INPUT_XLSX}: {e}")
        df_inputs = pd.DataFrame(fallback_data, columns=columns)
else:
    df_inputs = pd.DataFrame(fallback_data, columns=columns)

if SPRINT_WEEKEND:
    df_inputs["Proj Pts (Event)"] = (df_inputs["Avg Pts/GP"] * SPRINT_UPLIFT).round(2)
else:
    df_inputs["Proj Pts (Event)"] = df_inputs["Avg Pts/GP"].round(2)

# Merge VEPI (ajuste percentual por desvio-padrão)
if not vepi.empty:
    vepi_norm = vepi[["VEPI"]].copy()
    vepi_norm["VEPI_z"] = (vepi_norm["VEPI"] - vepi_norm["VEPI"].mean()) / (vepi_norm["VEPI"].std(ddof=0) + 1e-6)
    vepi_norm["VEPI_adj"] = 1 + (vepi_norm["VEPI_z"] * 0.10)
    merged = df_inputs.merge(vepi_norm[["VEPI_adj"]], left_on="Driver", right_index=True, how="left")
    merged["VEPI_adj"].fillna(1.0, inplace=True)
else:
    merged = df_inputs.copy()
    merged["VEPI_adj"] = 1.0

merged["Projected Vegas Pts (no mult)"] = (merged["Proj Pts (Event)"] * merged["VEPI_adj"]).round(2)
merged.head(8)"""))

cells.append(nbf.v4.new_code_cell("""
# =============
# 4) Mapear times e configurar construtores
# =============
# Mapeamento leve 2025 (ajuste se necessário)
TEAM_BY_DRIVER = {
    "Lando Norris": "McLaren",
    "Oscar Piastri": "McLaren",
    "Max Verstappen": "Red Bull Racing",
    "Liam Lawson": "Red Bull Racing",
    "George Russell": "Mercedes",
    "Kimi Antonelli": "Mercedes",
    "Lewis Hamilton": "Ferrari",
    "Charles Leclerc": "Ferrari",
    "Alexander Albon": "Williams",
    "Franco Colapinto": "Williams",
    "Nico Hülkenberg": "Kick Sauber",
    "Oliver Bearman": "Haas",  # ajustar se necessário
    "Esteban Ocon": "Alpine",
    "Pierre Gasly": "Alpine",
    "Isack Hadjar": "RB",
    "Yuki Tsunoda": "RB",
    "Lance Stroll": "Aston Martin",
    "Fernando Alonso": "Aston Martin",
    "Gabriel Bortoleto": "???",
    "Carlos Sainz": "???",
    "Jack Doohan": "Alpine",
}

merged["Team"] = merged["Driver"].map(TEAM_BY_DRIVER).fillna("Unknown")

# Parâmetros de pit dos construtores (faixas e bônus). Ajuste conforme seu modelo.
CONSTRUCTOR_COSTS = {
    "McLaren": 35.6,
    "Red Bull Racing": 30.2,
    "Ferrari": 33.0,
    "Mercedes": 32.0,
    "Williams": 18.0,
    "Aston Martin": 20.0,
    "Alpine": 15.0,
    "RB": 14.0,
    "Kick Sauber": 12.0,
    "Haas": 11.0,
    "Unknown": 10.0,
}

# Estimativas de pit para Vegas (pode substituir por valores reais quando tiver)
# Base + bônus (fastest 5, world record 15)
CONSTRUCTOR_PIT_POINTS = {
    "McLaren": {"base": 10, "fastest": 5, "world_record": 0},
    "Red Bull Racing": {"base": 5, "fastest": 0, "world_record": 0},
    "Ferrari": {"base": 5, "fastest": 0, "world_record": 0},
    "Mercedes": {"base": 2, "fastest": 0, "world_record": 0},
    "Williams": {"base": 0, "fastest": 0, "world_record": 0},
    "Aston Martin": {"base": 2, "fastest": 0, "world_record": 0},
    "Alpine": {"base": 0, "fastest": 0, "world_record": 0},
    "RB": {"base": 2, "fastest": 0, "world_record": 0},
    "Kick Sauber": {"base": 0, "fastest": 0, "world_record": 0},
    "Haas": {"base": 0, "fastest": 0, "world_record": 0},
    "Unknown": {"base": 0, "fastest": 0, "world_record": 0},
}

# Pontos estimados do construtor = soma(pts dos dois pilotos da equipe) + pit points
constructor_scores = []
for team, cost in CONSTRUCTOR_COSTS.items():
    team_drivers = merged[merged["Team"] == team]
    # Soma dos 2 melhores (ou de todos da equipe se 2 exatamente)
    top2 = team_drivers.nlargest(2, "Projected Vegas Pts (no mult)")
    drivers_sum = top2["Projected Vegas Pts (no mult)"].sum()
    pit = CONSTRUCTOR_PIT_POINTS.get(team, {"base":0,"fastest":0,"world_record":0})
    pit_points = pit["base"] + pit["fastest"] + pit["world_record"]
    constructor_scores.append((team, cost, drivers_sum, pit_points, drivers_sum + pit_points))

df_constructors = pd.DataFrame(constructor_scores, columns=["Constructor","Cost ($M)","Sum Drivers Pts","Pit Pts","Constructor Pts (Est.)"])

# Aplicar locks/exclusions para construtores
if LOCK_CONSTRUCTORS:
    df_constructors = df_constructors[df_constructors["Constructor"].isin(LOCK_CONSTRUCTORS + [c for c in df_constructors["Constructor"] if c not in EXCLUDE_CONSTRUCTORS])]
if EXCLUDE_CONSTRUCTORS:
    df_constructors = df_constructors[~df_constructors["Constructor"].isin(EXCLUDE_CONSTRUCTORS)]

df_constructors.sort_values("Constructor Pts (Est.)", ascending=False).head(5)"""))

cells.append(nbf.v4.new_code_cell("""
# =============
# 5) Otimizador com 5 pilotos + N construtores
# =============
cand = merged.dropna(subset=["Projected Vegas Pts (no mult)","Cost ($M)"]).copy()
cand = cand[cand["Cost ($M)"] > 0]

# Aplicar locks/exclusions de pilotos
if LOCK_DRIVERS:
    cand = cand[cand["Driver"].isin(LOCK_DRIVERS + [d for d in cand["Driver"] if d not in EXCLUDE_DRIVERS])]
if EXCLUDE_DRIVERS:
    cand = cand[~cand["Driver"].isin(EXCLUDE_DRIVERS)]

# Multiplicadores
def apply_mult(driver, pts):
    mult = DRIVER_MULTIPLIERS.get(driver, 1.0)
    return pts * mult

best = {"pts": -1, "drivers": None, "constructors": None, "cost": None}

driver_rows = list(cand.itertuples(index=False))
constructors_rows = list(df_constructors.itertuples(index=False))

def constructors_combos(rows, k):
    if k == 1:
        for r in rows:
            yield (r,)
    else:
        from itertools import combinations
        for combo in combinations(rows, k):
            yield combo

for drv_combo in combinations(driver_rows, 5):
    drivers_cost = sum(getattr(d, "_2") for d in drv_combo)  # Cost ($M)
    # early prune by budget (min constructor cost)
    min_const_cost = min([getattr(c, "_2") for c in constructors_rows]) * NUM_CONSTRUCTORS if constructors_rows else 0
    if drivers_cost + min_const_cost > TOTAL_BUDGET:
        continue

    drivers_pts = sum(apply_mult(d[0], getattr(d, "_6")) for d in drv_combo)  # Projected Vegas Pts (no mult) * mult

    for cons_combo in constructors_combos(constructors_rows, NUM_CONSTRUCTORS):
        cons_cost = sum(getattr(c, "_2") for c in cons_combo)
        total_cost = drivers_cost + cons_cost
        if total_cost > TOTAL_BUDGET:
            continue
        cons_pts = sum(getattr(c, "_5") for c in cons_combo)  # Constructor Pts (Est.)
        total_pts = drivers_pts + cons_pts
        if total_pts > best["pts"]:
            best = {"pts": total_pts, "drivers": drv_combo, "constructors": cons_combo, "cost": total_cost}

best_summary = None
if best["drivers"]:
    best_drivers_df = pd.DataFrame([{
        "Driver": d[0],
        "Team": d[8],
        "Cost ($M)": d[1],
        "Projected Vegas Pts (no mult)": d[6],
        "Multiplier": DRIVER_MULTIPLIERS.get(d[0], 1.0),
        "Projected Vegas Pts (with mult)": apply_mult(d[0], d[6]),
        "Value (pts/$M)": round(apply_mult(d[0], d[6]) / d[1], 3)
    } for d in best["drivers"]]).sort_values("Projected Vegas Pts (with mult)", ascending=False).reset_index(drop=True)

    best_constructors_df = pd.DataFrame([{
        "Constructor": c[0],
        "Cost ($M)": c[1],
        "Sum Drivers Pts": c[2],
        "Pit Pts": c[3],
        "Constructor Pts (Est.)": c[4]
    } for c in best["constructors"]]).sort_values("Constructor Pts (Est.)", ascending=False).reset_index(drop=True)

    best_summary = {
        "Total Cost ($M)": round(best["cost"], 2),
        "Total Projected Pts": round(best["pts"], 2),
        "Drivers Cost": round(best_drivers_df["Cost ($M)"].sum(), 2),
        "Constructors Cost": round(best_constructors_df["Cost ($M)"].sum(), 2),
        "Drivers Pts (with mult)": round(best_drivers_df["Projected Vegas Pts (with mult)"].sum(), 2),
        "Constructors Pts": round(best_constructors_df["Constructor Pts (Est.)"].sum(), 2),
    }

best_summary, best_drivers_df.head(10) if best_summary else None, best_constructors_df if best_summary else None"""))

cells.append(nbf.v4.new_code_cell("""
# =============
# 6) Exportar Excel v2
# =============
with pd.ExcelWriter(OUT_EXCEL, engine="xlsxwriter") as writer:
    merged.sort_values("Projected Vegas Pts (no mult)", ascending=False).to_excel(writer, sheet_name="Drivers Projections", index=False)
    if 'vepi' in globals() and not vepi.empty:
        vepi.reset_index().to_excel(writer, sheet_name="VEPI", index=False)
    df_constructors.sort_values("Constructor Pts (Est.)", ascending=False).to_excel(writer, sheet_name="Constructors", index=False)
    if best_summary:
        best_drivers_df.to_excel(writer, sheet_name="Best Drivers", index=False)
        best_constructors_df.to_excel(writer, sheet_name="Best Constructors", index=False)
        pd.DataFrame([best_summary]).to_excel(writer, sheet_name="Best Summary", index=False)

OUT_EXCEL"""))

nb['cells'] = cells

ipynb_path = Path("C:/Users/wxambinho/Documents/F1_Fantasy_Vegas_Model_v2.ipynb")
with open(ipynb_path, "w", encoding="utf-8") as f:
    nbf.write(nb, f)

print(f"Notebook salvo em: {ipynb_path}")



FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\wxambinho\\Documents\\F1_Fantasy_Vegas_Model_v2.ipynb'

In [14]:
RACE_TRACES_DIR = "/home/wxambinho/f1_race_traces_2021/data/laps_2025"

In [16]:
RACE_TRACES_DIR = "/home/wxambinho/f1_race_traces_2021/data/laps_2025"

In [15]:
RACE_TRACES_DIR = "/home/wxambinho/f1_race_traces_2021/data/laps_2025"

In [13]:
import os, time, unicodedata
import fastf1
import pandas as pd

# =========================
# Configurações e cache
# =========================
BASE_DIR = os.getcwd()
CACHE_DIR = os.path.join(BASE_DIR, "f1_race_traces_2021", "data", "fastf1_cache")
OUT_DIR   = os.path.join(BASE_DIR, "f1_race_traces_2021", "data", "laps_2025")

os.makedirs(CACHE_DIR, exist_ok=True)
os.makedirs(OUT_DIR, exist_ok=True)
fastf1.Cache.enable_cache(CACHE_DIR)

YEAR = 2025

# =========================
# Lista passada pelo usuário (nomes "oficiais"/promocionais)
# =========================
user_official_names = [
    "FORMULA 1 LOUIS VUITTON AUSTRALIAN GRAND PRIX 2025",
    "FORMULA 1 HEINEKEN CHINESE GRAND PRIX 2025",
    "FORMULA 1 LENOVO JAPANESE GRAND PRIX 2025",
    "FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2025",
    "FORMULA 1 STC SAUDI ARABIAN GRAND PRIX 2025",
    "FORMULA 1 CRYPTO.COM MIAMI GRAND PRIX 2025",
    "FORMULA 1 AWS GRAN PREMIO DEL MADE IN ITALY E DELL'EMILIA-ROMAGNA 2025",
    "FORMULA 1 TAG HEUER GRAND PRIX DE MONACO 2025",
    "FORMULA 1 ARAMCO GRAN PREMIO DE ESPAÑA 2025",
    "FORMULA 1 PIRELLI GRAND PRIX DU CANADA 2025",
    "FORMULA 1 MSC CRUISES AUSTRIAN GRAND PRIX 2025",
    "FORMULA 1 QATAR AIRWAYS BRITISH GRAND PRIX 2025",
    "FORMULA 1 MOËT & CHANDON BELGIAN GRAND PRIX 2025",
    "FORMULA 1 LENOVO HUNGARIAN GRAND PRIX 2025",
    "FORMULA 1 HEINEKEN DUTCH GRAND PRIX 2025",
    "FORMULA 1 PIRELLI GRAN PREMIO D’ITALIA 2025",
    "FORMULA 1 QATAR AIRWAYS AZERBAIJAN GRAND PRIX 2025",
    "FORMULA 1 SINGAPORE AIRLINES SINGAPORE GRAND PRIX 2025",
    "FORMULA 1 MSC CRUISES UNITED STATES GRAND PRIX 2025",
    "FORMULA 1 GRAN PREMIO DE LA CIUDAD DE MÉXICO 2025",
    "FORMULA 1 MSC CRUISES GRANDE PRÊMIO DE SÃO PAULO 2025",
    "FORMULA 1 HEINEKEN LAS VEGAS GRAND PRIX 2025",
    "FORMULA 1 QATAR AIRWAYS QATAR GRAND PRIX 2025",
    "FORMULA 1 ETIHAD AIRWAYS ABU DHABI GRAND PRIX 2025",
]

# =========================
# Funções auxiliares
# =========================
def normalize_text(s: str) -> str:
    """Normaliza: minúsculas, remove acentos e caracteres não alfanuméricos."""
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = s.lower()
    # mantém apenas letras e números, substitui por espaço
    out = []
    for ch in s:
        if ch.isalnum():
            out.append(ch)
        else:
            out.append(" ")
    # colapsa espaços
    norm = " ".join("".join(out).split())
    return norm

def best_match_from_schedule(target_name_norm: str, sched_df: pd.DataFrame):
    """
    Tenta casar 'target_name_norm' com colunas de nomes do calendário do FastF1.
    Retorna (round_number, event_name) ou None se não encontrar.
    Heurística:
      1) match exato com EventName normalizado
      2) match exato com OfficialEventName/FullEventName (se existirem)
      3) contain/startswith em qualquer um dos nomes conhecidos
      4) melhor candidato por similaridade simples (contagem de tokens)
    """
    # Colunas possíveis (variam por versão do FastF1)
    name_cols = []
    for cand in ["EventName", "OfficialEventName", "EventOfficialName", "FullEventName", "EventShortName"]:
        if cand in sched_df.columns:
            name_cols.append(cand)
    if "RoundNumber" not in sched_df.columns:
        raise RuntimeError("Calendário do FastF1 não tem RoundNumber; versão incomum do pacote.")

    # Prepara tabela com colunas normalizadas
    rows = []
    for _, row in sched_df.iterrows():
        names = {}
        for c in name_cols:
            val = str(row[c])
            names[c] = val
        # normalizações
        norms = {c: normalize_text(v) for c, v in names.items()}
        rows.append({
            "round": int(row["RoundNumber"]),
            "names": names,
            "norms": norms
        })

    # 1) match exato
    for r in rows:
        if any(target_name_norm == n for n in r["norms"].values()):
            # prioriza usar EventName "sem patrocínio" se existir
            event_name = r["names"].get("EventName") or list(r["names"].values())[0]
            return r["round"], event_name

    # 2) contain/startswith
    for r in rows:
        if any(target_name_norm in n or n in target_name_norm for n in r["norms"].values()):
            event_name = r["names"].get("EventName") or list(r["names"].values())[0]
            return r["round"], event_name

    # 3) scoring simples por interseção de tokens
    tokens_t = set(target_name_norm.split())
    best = None
    best_score = 0
    for r in rows:
        cand_tokens = set(" ".join(r["norms"].values()).split())
        score = len(tokens_t.intersection(cand_tokens))
        if score > best_score:
            best = r
            best_score = score
    if best and best_score >= 2:  # exige interseção mínima
        event_name = best["names"].get("EventName") or list(best["names"].values())[0]
        return best["round"], event_name

    return None

# =========================
# Carrega calendário 2025
# =========================
schedule = fastf1.get_event_schedule(YEAR)  # DataFrame
print(f"Calendário {YEAR}: {len(schedule)} eventos\n")
if len(schedule) == 0:
    raise RuntimeError("Calendário vazio. Verifique sua conexão ou a versão do FastF1.")

# Mostra preview dos nomes que o FastF1 conhece (ajuda na depuração)
cols_preview = [c for c in ["RoundNumber","EventName","OfficialEventName","EventOfficialName","FullEventName","EventShortName","EventDate"] if c in schedule.columns]
display(schedule[cols_preview].head(10))

# =========================
# Resolve e baixa corridas
# =========================
success = []
skipped = []
errors  = []

for user_name in user_official_names:
    target_norm = normalize_text(user_name)
    resolved = best_match_from_schedule(target_norm, schedule)

    if not resolved:
        print(f"❓ Não consegui resolver o nome: {user_name}")
        skipped.append((user_name, "no-match"))
        continue

    round_no, event_name = resolved
    print(f"\n=== {user_name} → FastF1: Round {round_no} | EventName='{event_name}' ===")
    try:
        session = fastf1.get_session(YEAR, round_no, "R")  # usa o round: mais confiável
        session.load()  # baixa/usa cache
        laps = session.laps

        if laps.empty:
            print(f"⚠️  Sem dados de corrida disponíveis ainda para '{event_name}'.")
            skipped.append((user_name, "empty"))
            continue

        safe_gp = event_name.replace(" ", "_")
        out_csv = os.path.join(OUT_DIR, f"laps_{YEAR}_{safe_gp}.csv")
        cols = ["Driver", "LapNumber", "LapTime", "Compound", "Stint", "PitInTime", "PitOutTime"]
        (laps[cols] if all(c in laps.columns for c in cols) else laps).to_csv(out_csv, index=False)
        print(f"✅ Salvo: {out_csv} ({len(laps)} voltas)")
        success.append((user_name, event_name, round_no, len(laps)))

        time.sleep(1.5)  # boa prática entre requisições

    except Exception as e:
        print(f"❌ Erro ao baixar '{event_name}': {e}")
        errors.append((user_name, str(e)))

# =========================
# Resumo
# =========================
print("\n=== RESUMO ===")
print(f"OK: {len(success)} | Skipped: {len(skipped)} | Erros: {len(errors)}")
if success:
    df_ok = pd.DataFrame(success, columns=["UserName","EventName","Round","TotalLaps"])
    display(df_ok.sort_values("Round"))
if skipped:
    df_sk = pd.DataFrame(skipped, columns=["UserName","Reason"])
    print("\nSkipped:")
    display(df_sk)
if errors:
    df_er = pd.DataFrame(errors, columns=["UserName","Error"])
    print("\nErros:")
    display(df_er)

print(f"\nArquivos gerados em: {OUT_DIR}")


Calendário 2025: 25 eventos



,RoundNumber,EventName,OfficialEventName,EventDate
0,0,Pre-Season Testing,FORMULA 1 ARAMCO PRE-SEASON TESTING 2025,2025-02-28
1,1,Australian Grand Prix,FORMULA 1 LOUIS VUITTON AUSTRALIAN GRAND PRIX ...,2025-03-16
2,2,Chinese Grand Prix,FORMULA 1 HEINEKEN CHINESE GRAND PRIX 2025,2025-03-23
3,3,Japanese Grand Prix,FORMULA 1 LENOVO JAPANESE GRAND PRIX 2025,2025-04-06
4,4,Bahrain Grand Prix,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2025,2025-04-13
5,5,Saudi Arabian Grand Prix,FORMULA 1 STC SAUDI ARABIAN GRAND PRIX 2025,2025-04-20
6,6,Miami Grand Prix,FORMULA 1 CRYPTO.COM MIAMI GRAND PRIX 2025,2025-05-04
7,7,Emilia Romagna Grand Prix,FORMULA 1 AWS GRAN PREMIO DEL MADE IN ITALY E ...,2025-05-18
8,8,Monaco Grand Prix,FORMULA 1 TAG HEUER GRAND PRIX DE MONACO 2025,2025-05-25
9,9,Spanish Grand Prix,FORMULA 1 ARAMCO GRAN PREMIO DE ESPAÑA 2025,2025-06-01


core           INFO 	Loading data for Australian Grand Prix - Race [v3.6.1]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...



=== FORMULA 1 LOUIS VUITTON AUSTRALIAN GRAND PRIX 2025 → FastF1: Round 1 | EventName='Australian Grand Prix' ===


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!

...

❌ Erro ao baixar 'Abu Dhabi Grand Prix': The data you are trying to access has not been loaded yet. See `Session.load`

=== RESUMO ===
OK: 21 | Skipped: 0 | Erros: 3


,UserName,EventName,Round,TotalLaps
0,FORMULA 1 LOUIS VUITTON AUSTRALIAN GRAND PRIX ...,Australian Grand Prix,1,927
1,FORMULA 1 HEINEKEN CHINESE GRAND PRIX 2025,Chinese Grand Prix,2,1065
2,FORMULA 1 LENOVO JAPANESE GRAND PRIX 2025,Japanese Grand Prix,3,1059
3,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2025,Bahrain Grand Prix,4,1128
4,FORMULA 1 STC SAUDI ARABIAN GRAND PRIX 2025,Saudi Arabian Grand Prix,5,898
5,FORMULA 1 CRYPTO.COM MIAMI GRAND PRIX 2025,Miami Grand Prix,6,1005
6,FORMULA 1 AWS GRAN PREMIO DEL MADE IN ITALY E ...,Emilia Romagna Grand Prix,7,1207
7,FORMULA 1 TAG HEUER GRAND PRIX DE MONACO 2025,Monaco Grand Prix,8,1425
8,FORMULA 1 ARAMCO GRAN PREMIO DE ESPAÑA 2025,Spanish Grand Prix,9,1203
9,FORMULA 1 PIRELLI GRAND PRIX DU CANADA 2025,Canadian Grand Prix,10,1349



Erros:


,UserName,Error
0,FORMULA 1 HEINEKEN LAS VEGAS GRAND PRIX 2025,The data you are trying to access has not been...
1,FORMULA 1 QATAR AIRWAYS QATAR GRAND PRIX 2025,The data you are trying to access has not been...
2,FORMULA 1 ETIHAD AIRWAYS ABU DHABI GRAND PRIX ...,The data you are trying to access has not been...



Arquivos gerados em: C:\F1\setup\f1_race_traces_2021\f1_race_traces_2021\data\laps_2025


In [12]:
import fastf1, os, pandas as pd, time

# 🏁 Ativar cache local (acelera execuções futuras)
cache_dir = os.path.join(os.getcwd(), "f1_race_traces_2021", "data", "fastf1_cache")
os.makedirs(cache_dir, exist_ok=True)
fastf1.Cache.enable_cache(cache_dir)

# 📅 Obter calendário oficial de 2025
schedule = fastf1.get_event_schedule(2025)
print("Total de eventos no calendário:", len(schedule))

# 📂 Pasta para salvar os CSVs
out_dir = os.path.join(os.getcwd(), "f1_race_traces_2021", "data", "laps_2025")
os.makedirs(out_dir, exist_ok=True)

# 🚗 Loop para baixar cada corrida (sessão "R" = Race)
for _, event in schedule.iterrows():
    gp_name = event['EventName']
    year = event['Year']
    try:
        print(f"\n=== Baixando {gp_name} {year} ===")
        session = fastf1.get_session(year, gp_name, "R")
        session.load()
        laps = session.laps
        if laps.empty:
            print(f"⚠️  Nenhum dado disponível ainda para {gp_name}.")
            continue

        # Salva um CSV resumido com as voltas
        out_csv = os.path.join(out_dir, f"laps_{year}_{gp_name.replace(' ', '_')}.csv")
        laps[['Driver', 'LapNumber', 'LapTime', 'Compound', 'Stint', 'PitInTime', 'PitOutTime']].to_csv(out_csv, index=False)
        print(f"✅ Salvo: {out_csv} ({len(laps)} voltas)")

        # pausa pequena entre downloads (boas práticas com a API)
        time.sleep(2)

    except Exception as e:
        print(f"❌ Erro em {gp_name}: {e}")

print("\n=== Concluído! Verifique os arquivos em ===")
print(out_dir)



Total de eventos no calendário: 25


KeyError: 'Year'

In [ ]:
import fastf1, os

# ativa cache local
cache_dir = os.path.join(os.getcwd(), "f1_race_traces_2021", "data", "fastf1_cache")
os.makedirs(cache_dir, exist_ok=True)
fastf1.Cache.enable_cache(cache_dir)

# Exemplo: corrida da Arábia Saudita 2025 (ajuste o nome do GP conforme o calendário real)
session = fastf1.get_session(2025, "FORMULA 1 ARAMCO PRE - SEASON", "R")
session.load()

laps = session.laps
print(f"Total de voltas registradas: {len(laps)}")
print("Pilotos:", laps['Driver'].unique())


In [9]:
fastf1.get_event_schedule(2025)

,RoundNumber,Country,Location,OfficialEventName,EventDate,EventName,EventFormat,Session1,Session1Date,Session1DateUtc,...,Session3,Session3Date,Session3DateUtc,Session4,Session4Date,Session4DateUtc,Session5,Session5Date,Session5DateUtc,F1ApiSupport
0,0,Bahrain,Sakhir,FORMULA 1 ARAMCO PRE-SEASON TESTING 2025,2025-02-28,Pre-Season Testing,testing,Practice 1,2025-02-26 10:00:00+03:00,2025-02-26 07:00:00,...,Practice 3,2025-02-28 10:00:00+03:00,2025-02-28 07:00:00,None,NaT,NaT,None,NaT,NaT,True
1,1,Australia,Melbourne,FORMULA 1 LOUIS VUITTON AUSTRALIAN GRAND PRIX ...,2025-03-16,Australian Grand Prix,conventional,Practice 1,2025-03-14 12:30:00+11:00,2025-03-14 01:30:00,...,Practice 3,2025-03-15 12:30:00+11:00,2025-03-15 01:30:00,Qualifying,2025-03-15 16:00:00+11:00,2025-03-15 05:00:00,Race,2025-03-16 15:00:00+11:00,2025-03-16 04:00:00,True
2,2,China,Shanghai,FORMULA 1 HEINEKEN CHINESE GRAND PRIX 2025,2025-03-23,Chinese Grand Prix,sprint_qualifying,Practice 1,2025-03-21 11:30:00+08:00,2025-03-21 03:30:00,...,Sprint,2025-03-22 11:00:00+08:00,2025-03-22 03:00:00,Qualifying,2025-03-22 15:00:00+08:00,2025-03-22 07:00:00,Race,2025-03-23 15:00:00+08:00,2025-03-23 07:00:00,True
3,3,Japan,Suzuka,FORMULA 1 LENOVO JAPANESE GRAND PRIX 2025,2025-04-06,Japanese Grand Prix,conventional,Practice 1,2025-04-04 11:30:00+09:00,2025-04-04 02:30:00,...,Practice 3,2025-04-05 11:30:00+09:00,2025-04-05 02:30:00,Qualifying,2025-04-05 15:00:00+09:00,2025-04-05 06:00:00,Race,2025-04-06 14:00:00+09:00,2025-04-06 05:00:00,True
4,4,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2025,2025-04-13,Bahrain Grand Prix,conventional,Practice 1,2025-04-11 14:30:00+03:00,2025-04-11 11:30:00,...,Practice 3,2025-04-12 15:30:00+03:00,2025-04-12 12:30:00,Qualifying,2025-04-12 19:00:00+03:00,2025-04-12 16:00:00,Race,2025-04-13 18:00:00+03:00,2025-04-13 15:00:00,True
5,5,Saudi Arabia,Jeddah,FORMULA 1 STC SAUDI ARABIAN GRAND PRIX 2025,2025-04-20,Saudi Arabian Grand Prix,conventional,Practice 1,2025-04-18 16:30:00+03:00,2025-04-18 13:30:00,...,Practice 3,2025-04-19 16:30:00+03:00,2025-04-19 13:30:00,Qualifying,2025-04-19 20:00:00+03:00,2025-04-19 17:00:00,Race,2025-04-20 20:00:00+03:00,2025-04-20 17:00:00,True
6,6,United States,Miami Gardens,FORMULA 1 CRYPTO.COM MIAMI GRAND PRIX 2025,2025-05-04,Miami Grand Prix,sprint_qualifying,Practice 1,2025-05-02 12:30:00-04:00,2025-05-02 16:30:00,...,Sprint,2025-05-03 12:00:00-04:00,2025-05-03 16:00:00,Qualifying,2025-05-03 16:00:00-04:00,2025-05-03 20:00:00,Race,2025-05-04 16:00:00-04:00,2025-05-04 20:00:00,True
7,7,Italy,Imola,FORMULA 1 AWS GRAN PREMIO DEL MADE IN ITALY E ...,2025-05-18,Emilia Romagna Grand Prix,conventional,Practice 1,2025-05-16 13:30:00+02:00,2025-05-16 11:30:00,...,Practice 3,2025-05-17 12:30:00+02:00,2025-05-17 10:30:00,Qualifying,2025-05-17 16:00:00+02:00,2025-05-17 14:00:00,Race,2025-05-18 15:00:00+02:00,2025-05-18 13:00:00,True
8,8,Monaco,Monaco,FORMULA 1 TAG HEUER GRAND PRIX DE MONACO 2025,2025-05-25,Monaco Grand Prix,conventional,Practice 1,2025-05-23 13:30:00+02:00,2025-05-23 11:30:00,...,Practice 3,2025-05-24 12:30:00+02:00,2025-05-24 10:30:00,Qualifying,2025-05-24 16:00:00+02:00,2025-05-24 14:00:00,Race,2025-05-25 15:00:00+02:00,2025-05-25 13:00:00,True
9,9,Spain,Barcelona,FORMULA 1 ARAMCO GRAN PREMIO DE ESPAÑA 2025,2025-06-01,Spanish Grand Prix,conventional,Practice 1,2025-05-30 13:30:00+02:00,2025-05-30 11:30:00,...,Practice 3,2025-05-31 12:30:00+02:00,2025-05-31 10:30:00,Qualifying,2025-05-31 16:00:00+02:00,2025-05-31 14:00:00,Race,2025-06-01 15:00:00+02:00,2025-06-01 13:00:00,True


In [7]:
import fastf1, os

cache_dir = os.path.join(os.getcwd(), "f1_race_traces_2021", "data", "fastf1_cache")
os.makedirs(cache_dir, exist_ok=True)
fastf1.Cache.enable_cache(cache_dir)

session = fastf1.get_session(2021, "Bahrain", "R")
session.load()

laps = session.laps
print("Pilotos:", sorted(laps['Driver'].unique())[:10], "…  (total:", laps['Driver'].nunique(), ")")
print("Total de voltas registradas:", len(laps))


core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.6.1]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No ca

Pilotos: ['ALO', 'BOT', 'GAS', 'GIO', 'HAM', 'LAT', 'LEC', 'MAZ', 'MSC', 'NOR'] …  (total: 20 )
Total de voltas registradas: 1027


<h1></h1>

This notebook visualizes race traces for every F1 Grand Prix in 2021, using historical data from the [Formula 1 Race Data](https://www.kaggle.com/datasets/jtrotman/formula-1-race-data) dataset (provided by [ergast.com](http://ergast.com/mrd/) and updated on Kaggle after every race), along with [race event information from Wikipedia](https://www.kaggle.com/jtrotman/formula-1-race-events).

Race traces are a great way to capture the dynamics of a Grand Prix.
They show the gaps between cars, how the field spreads out over time, and the relative pace of each car as the race progresses.
These traces are based on cumulative lap times, adjusted using the median lap time at each point in the race.
The horizontal zero line serves as a reference for a virtual car running at the field’s median average lap time—cars above this line are faster, while those below are slower.

Here are some key features of the race traces:

 - The line **colors** for each driver are based on their team colors.
 - The **fastest lap** is marked with a &#9733;.
 - Laps where a car went via the **pit lane** are marked with a &#9679;.
 - Laps affected by the **safety car** are highlighted in yellow, indicating why the gaps between cars have reduced.
 - **[Virtual safety cars][1]**, introduced in 2015, are highlighted in orange, as they cause the lines to spread out temporarily since leaders have completed more of the first affected lap at racing speeds.
 - **Overtakes** are visible where the lines cross, often due to pit-stops.
 - Truncated lines indicate **retirement** from the race.

The **shadow** at the bottom of the trace indicates which cars have been lapped by the lead car.
This helps to reveal some effects, such as leaders making pit-stops to avoid slower traffic ahead.
Backmarkers must allow leading cars through, and the edge of the shadow shows why their lines suddenly dip.
Leaders may also ease off to let a lapped car through at the end of a race.

It's worth noting that good safety car data is hard to find, and the data scraped from Wikipedia may contain some mistakes.
Therefore, the virtual safety car highlights in the traces are the author's estimates using lap-time data.

Overall, these race traces offer a comprehensive and informative way to understand the progress of each F1 race in 2021.

________

*Tip:* to see better resolution, &lt;*Right click*&gt; &rarr; *Open Image in New Tab*.


## Revisions

 * _version 1: fixed code for Melbourne 2021 postponement_
 * _version 5: add fastest lap marker_
 * _version 11: added laps per position and top drivers chart_
 * _version 21: safety car, red flag & lapped cars highlighting_
 * _version 22: add sprint race points to table at end_


## All F1 Race Traces

There is a notebook for every year that has lap-time data:

[1996](https://www.kaggle.com/code/jtrotman/f1-race-traces-1996), 
[1997](https://www.kaggle.com/code/jtrotman/f1-race-traces-1997), 
[1998](https://www.kaggle.com/code/jtrotman/f1-race-traces-1998), 
[1999](https://www.kaggle.com/code/jtrotman/f1-race-traces-1999), 
[2000](https://www.kaggle.com/code/jtrotman/f1-race-traces-2000), 
[2001](https://www.kaggle.com/code/jtrotman/f1-race-traces-2001), 
[2002](https://www.kaggle.com/code/jtrotman/f1-race-traces-2002), 
[2003](https://www.kaggle.com/code/jtrotman/f1-race-traces-2003), 
[2004](https://www.kaggle.com/code/jtrotman/f1-race-traces-2004), 
[2005](https://www.kaggle.com/code/jtrotman/f1-race-traces-2005), 
[2006](https://www.kaggle.com/code/jtrotman/f1-race-traces-2006), 
[2007](https://www.kaggle.com/code/jtrotman/f1-race-traces-2007), 
[2008](https://www.kaggle.com/code/jtrotman/f1-race-traces-2008), 
[2009](https://www.kaggle.com/code/jtrotman/f1-race-traces-2009), 
[2010](https://www.kaggle.com/code/jtrotman/f1-race-traces-2010), 
[2011](https://www.kaggle.com/code/jtrotman/f1-race-traces-2011), 
[2012](https://www.kaggle.com/code/jtrotman/f1-race-traces-2012), 
[2013](https://www.kaggle.com/code/jtrotman/f1-race-traces-2013), 
[2014](https://www.kaggle.com/code/jtrotman/f1-race-traces-2014), 
[2015](https://www.kaggle.com/code/jtrotman/f1-race-traces-2015), 
[2016](https://www.kaggle.com/code/jtrotman/f1-race-traces-2016), 
[2017](https://www.kaggle.com/code/jtrotman/f1-race-traces-2017), 
[2018](https://www.kaggle.com/code/jtrotman/f1-race-traces-2018), 
[2019](https://www.kaggle.com/code/jtrotman/f1-race-traces-2019), 
[2020](https://www.kaggle.com/code/jtrotman/f1-race-traces-2020), 
[2021](https://www.kaggle.com/code/jtrotman/f1-race-traces-2021), 
[2022](https://www.kaggle.com/code/jtrotman/f1-race-traces-2022), 
[2023](https://www.kaggle.com/code/jtrotman/f1-race-traces-2023), 
[2024](https://www.kaggle.com/code/jtrotman/f1-race-traces-2024), 
[2025](https://www.kaggle.com/code/jtrotman/f1-race-traces-2025).

 [1]: https://en.wikipedia.org/wiki/Safety_car#Virtual_safety_car_(VSC)
 [2]: https://www.kaggle.com/cjgdev/formula-1-race-data-19502017
 [3]: https://www.kaggle.com/jtrotman/formula-1-race-events

In [4]:
YEAR = 2021
# Driver IDs for 2021:
#     1 = Lewis Hamilton
#     4 = Fernando Alonso
#     8 = Kimi Räikkönen
#     9 = Robert Kubica
#    20 = Sebastian Vettel
#   815 = Sergio Pérez
#   817 = Daniel Ricciardo
#   822 = Valtteri Bottas
#   830 = Max Verstappen
#   832 = Carlos Sainz
#   839 = Esteban Ocon
#   840 = Lance Stroll
#   841 = Antonio Giovinazzi
#   842 = Pierre Gasly
#   844 = Charles Leclerc
#   846 = Lando Norris
#   847 = George Russell
#   849 = Nicholas Latifi
#   852 = Yuki Tsunoda
#   853 = Nikita Mazepin
#   854 = Mick Schumacher
# Team IDs for 2021:
#     1 = McLaren
#     3 = Williams
#     6 = Ferrari
#     9 = Red Bull
#    51 = Alfa Romeo
#   117 = Aston Martin
#   131 = Mercedes
#   210 = Haas F1 Team
#   213 = AlphaTauri
#   214 = Alpine F1 Team
DRIVER_LS = {1:0,4:0,8:0,9:2,20:0,815:1,817:1,822:1,830:0,832:0,839:1,840:1,841:1,842:0,844:1,846:0,847:0,849:1,852:1,853:0,854:1}
DRIVER_C = {1:"#00CACA",4:"#8A2BE2",8:"#800000",9:"#800000",20:"#2E8B57",815:"#0000B0",817:"#FE7F00",822:"#00CACA",830:"#0000B0",832:"#FF0000",839:"#8A2BE2",840:"#2E8B57",841:"#800000",842:"#7F7F7F",844:"#FF0000",846:"#FE7F00",847:"#007FFE",849:"#007FFE",852:"#7F7F7F",853:"#191919",854:"#191919"}
TEAM_C = {1:"#FE7F00",3:"#007FFE",6:"#FF0000",9:"#0000B0",51:"#800000",117:"#2E8B57",131:"#00CACA",210:"#191919",213:"#7F7F7F",214:"#8A2BE2"}
LINESTYLES = ['-', '-.', '--', ':', '-', '-']

## 2021 Formula One World Championship

For background see [Wikipedia](https://en.wikipedia.org/wiki/2021_Formula_One_World_Championship); here's an excerpt:

The **2021 FIA Formula One World Championship** was a [motor racing championship][1] for [Formula One cars][2] which was the 72nd running of the [Formula One World Championship][3].
It is recognised by the [Fédération Internationale de l'Automobile][4] \(FIA\), the governing body of international [motorsport][5], as the highest class of competition for [open-wheel racing cars][6].
The championship was contested over twenty-two [Grands Prix][7], and held around the world.
Drivers and teams competed for the titles of [World Drivers' Champion][8] and [World Constructors' Champion][9], respectively.

At season's end in Abu Dhabi, [Max Verstappen][10] of [Red Bull Racing][12]-[Honda][13] won the Drivers' Championship for the first time in his career.
Verstappen became the first ever [driver from the Netherlands][11],  the first Honda-powered driver since [Ayrton Senna][18] in [1991][19],  the first Red Bull driver since [Sebastian Vettel][20] in [2013][21] and the first non-Mercedes driver in the turbo-hybrid era to win the World Championship.

Honda became the second engine supplier in the turbo-hybrid era to power a championship winning car, after Mercedes.
Four-time defending and seven-time champion [Lewis Hamilton][14] of [Mercedes][16] finished runner up.
Mercedes retained the Constructors' Championship for the eighth consecutive season.

The season ended with a controversial finish, with the two title rivals for the drivers' crown entering the last race of the season with equal points.
Verstappen sealed the title after winning the season-ending [Abu Dhabi Grand Prix][22] after a last-lap restart pass on Hamilton following a contentious conclusion of a safety car period.
Mercedes initially protested the results, and later decided not to appeal after their protest was denied.
The incident led to key structural changes to race control, including the removal of [Michael Masi][23] from his role as race director and the implementation of a virtual race control room, who assist the race director.
Unlapping procedures behind the safety car were to be reassessed and presented by the F1 Sporting Advisory Committee prior to the start of the [2022 World Championship season][24].
On 10 March 2022 the FIA [World Motor Sport Council][25] report on the events of the final race of the season was announced, and that the "Race Director called the safety car back into the pit lane without it having completed an additional lap as required by the Formula 1 Sporting Regulations", however also noted that the "results of the 2021 Abu Dhabi Grand Prix and the FIA Formula One World Championship are valid, final and cannot now be changed".

This was the first season since [2008][26] where the champion driver was not from the team that took the constructors' title.


 [1]: https://en.wikipedia.org/wiki/List_of_motorsport_championships "List of motorsport championships"
 [2]: https://en.wikipedia.org/wiki/Formula_One_cars "Formula One cars"
 [3]: https://en.wikipedia.org/wiki/Formula_One_World_Championship "Formula One World Championship"
 [4]: https://en.wikipedia.org/wiki/F%C3%A9d%C3%A9ration_Internationale_de_l%27Automobile "Fédération Internationale de l'Automobile"
 [5]: https://en.wikipedia.org/wiki/Motorsport "Motorsport"
 [6]: https://en.wikipedia.org/wiki/Open-wheel_car "Open-wheel car"
 [7]: https://en.wikipedia.org/wiki/List_of_Formula_One_Grands_Prix "List of Formula One Grands Prix"
 [8]: https://en.wikipedia.org/wiki/List_of_Formula_One_World_Drivers%27_Champions "List of Formula One World Drivers' Champions"
 [9]: https://en.wikipedia.org/wiki/List_of_Formula_One_World_Constructors%27_Champions "List of Formula One World Constructors' Champions"
 [10]: https://en.wikipedia.org/wiki/Max_Verstappen "Max Verstappen"
 [11]: https://en.wikipedia.org/wiki/Formula_One_drivers_from_the_Netherlands "Formula One drivers from the Netherlands"
 [12]: https://en.wikipedia.org/wiki/Red_Bull_Racing "Red Bull Racing"
 [13]: https://en.wikipedia.org/wiki/Honda_in_Formula_One "Honda in Formula One"
 [14]: https://en.wikipedia.org/wiki/Lewis_Hamilton "Lewis Hamilton"
 [15]: https://en.wikipedia.org/wiki/2020_Formula_One_World_Championship "2020 Formula One World Championship"
 [16]: https://en.wikipedia.org/wiki/Mercedes-Benz_in_Formula_One "Mercedes-Benz in Formula One"
 [17]: https://en.wikipedia.org/wiki/Valtteri_Bottas "Valtteri Bottas"
 [18]: https://en.wikipedia.org/wiki/Ayrton_Senna "Ayrton Senna"
 [19]: https://en.wikipedia.org/wiki/1991_Formula_One_World_Championship "1991 Formula One World Championship"
 [20]: https://en.wikipedia.org/wiki/Sebastian_Vettel "Sebastian Vettel"
 [21]: https://en.wikipedia.org/wiki/2013_Formula_One_World_Championship "2013 Formula One World Championship"
 [22]: https://en.wikipedia.org/wiki/2021_Abu_Dhabi_Grand_Prix "2021 Abu Dhabi Grand Prix"
 [23]: https://en.wikipedia.org/wiki/Michael_Masi "Michael Masi"
 [24]: https://en.wikipedia.org/wiki/2022_Formula_One_World_Championship "2022 Formula One World Championship"
 [25]: https://en.wikipedia.org/wiki/World_Motor_Sport_Council "World Motor Sport Council"
 [26]: https://en.wikipedia.org/wiki/2008_Formula_One_World_Championship "2008 Formula One World Championship"

In [5]:
from IPython.display import HTML, display
HTML("""<div id="contents"></div>
<script>
function fill_toc() {
  l = document.querySelectorAll("h2[id^=race]")
  src = '<h2 id="index-races">Races</h2>'
  for (const e of l) src += `<li><a href="#${e.id}">${e.textContent}</a>`;
  document.querySelector("#contents").innerHTML = src;
}
</script>""")

In [6]:
import base64, io, json, os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import urllib
from collections import Counter
import warnings
warnings.simplefilter("ignore")


def read_csv(name, **kwargs):
    df = pd.read_csv(f'../input/formula-1-race-data/{name}', na_values=r'\N', **kwargs)
    return df

# create replacement raceId that is ordered by time
def race_key(race_year, race_round):
    # My bet: we won't get to 100 races per year :)
    return (race_year * 100) + race_round

def races_subset(df, race_ids):
    df = df[df.raceId.isin(race_ids)].copy()
    df = df.join(races[['round', 'raceKey']], on='raceId')
    df['round'] -= df['round'].min()
    # drop_duplicates: duplicate entries have appeared in 2021
    # race:1051 driver:832
    return df.set_index('round').sort_index().drop_duplicates()

def add_lap_0(df):
    copy = df.T
    copy.insert(0, 0, 0)
    return copy.T

def driver_tag(driver_df_row):
    return ('<a href="{url}" title="Number: {number:.0f}\n'
            'Nationality: {nationality}">{Driver}</a>').format(**driver_df_row)

def constructor_tag(constructor_df_row):
    return ('<a href="{url}" title="Nationality: {nationality}">'
            '{name}</a>').format(**constructor_df_row)


RACE_COLS = ['raceId', 'year', 'round', 'circuitId', 'name', 'date', 'time', 'url']

# Read data
circuits = read_csv('circuits.csv', index_col=0)
constructorResults = read_csv('constructor_results.csv', index_col=0)
constructors = read_csv('constructors.csv', index_col=0)
constructorStandings = read_csv('constructor_standings.csv', index_col=0)
drivers = read_csv('drivers.csv', index_col=0)
driverStandings = read_csv('driver_standings.csv', index_col=0)
lapTimes = read_csv('lap_times.csv')
pitStops = read_csv('pit_stops.csv')
qualifying = read_csv('qualifying.csv', index_col=0)
races = read_csv('races.csv', index_col='raceId', usecols=RACE_COLS)
results = read_csv('results.csv', index_col=0)
seasons = read_csv('seasons.csv', index_col=0)
status = read_csv('status.csv', index_col=0)

# Additional dataset:
# https://www.kaggle.com/jtrotman/formula-1-race-events
safety_cars = pd.read_csv('../input/formula-1-race-events/safety_cars.csv')
red_flags = pd.read_csv('../input/formula-1-race-events/red_flags.csv')
with open('../input/formula-1-race-events/virtual_safety_car_estimates.json') as f:
    virtual_safety_cars = json.load(f)

# Additional dataset:
# https://www.kaggle.com/jtrotman/formula-1-pitstops-1994-2010
if os.path.isdir('../input/formula-1-pitstops-1994-2010'):
    old_pitstops = pd.read_csv('../input/formula-1-pitstops-1994-2010/pitstops.csv')
    pitStops = pd.concat(
        (pitStops,
         old_pitstops.rename(columns={
             'RaceId': 'raceId',
             'DriverId': 'driverId',
             'Lap': 'lap',
             'Stops': 'stop'
         })[['raceId', 'driverId', 'lap', 'stop']]),
        ignore_index=True)

# To sequence the races if they did not happen in order of raceId (ie. 2021)
races['raceKey'] = race_key(races['year'], races['round'])

# For display in HTML tables
drivers['display'] = drivers.surname
drivers['Driver'] = drivers['forename'] + " " + drivers['surname']
drivers['Driver'] = drivers.apply(driver_tag, axis=1)
constructors['label'] = constructors['name']
constructors['name'] = constructors.apply(constructor_tag, axis=1)

# Join fields
results['status'] = results.statusId.map(status.status)
results['Team'] = results.constructorId.map(constructors.name)
results['score'] = results.points>0
results['podium'] = results.position<=3

# Cut data to one year
# was using numexpr here but 2.8.5 breaks pandas:
# https://github.com/pandas-dev/pandas/issues/54449
races = races.loc[races.year==YEAR].sort_values('round').copy()
results = results[results.raceId.isin(races.index)].copy()
lapTimes = lapTimes[lapTimes.raceId.isin(races.index)].copy()
# Save Ids of races that have actually happened (i.e. have valid lap-times).
race_ids = np.unique(lapTimes.raceId)
driverStandings = races_subset(driverStandings, race_ids)
constructorStandings = races_subset(constructorStandings, race_ids)

# Sprint results (2021 onwards)
if os.path.isfile('../input/formula-1-race-data/sprint_results.csv'):
    sprint_results = pd.read_csv('../input/formula-1-race-data/sprint_results.csv', na_values=r'\N')
    sprint_results = sprint_results[sprint_results.raceId.isin(races.index)].copy()

lapTimes = lapTimes.merge(results[['raceId', 'driverId', 'positionOrder']], on=['raceId', 'driverId'])
lapTimes['seconds'] = lapTimes.pop('milliseconds') / 1000

def formatter(v):
    if type(v) is str:
        return v
    if pd.isna(v) or v <= 0:
        return ''
    if v == int(v):
        return f'{v:.0f}'
    return f'{v:.1f}'

def table_html(table, caption):
    return (f'<h3>{caption}</h3>' +
            table.style.format(formatter).to_html())

# Processing for Drivers & Constructors championship tables
def format_standings(df, key):
    df = df.sort_values('position')
    gb = results.groupby(key)
    df['Position'] = df.positionText
    df['scores'] = gb.score.sum()
    df['podiums'] = gb.podium.sum()
    return df

# Drivers championship table
def drivers_standings(df):
    index = 'driverId'
    df = df.set_index(index)
    df = df.join(drivers)
    df = format_standings(df, index)
    df['Team'] = results.groupby(index).Team.last()
    use = ['Position', 'Driver',  'Team', 'points', 'wins', 'podiums', 'scores', 'nationality' ]
    df = df[use].set_index('Position')
    df.columns = df.columns.str.capitalize()
    return df

# Constructors championship table
def constructors_standings(df):
    index = 'constructorId'
    df = df.set_index(index)
    df = df.join(constructors)
    df = format_standings(df, index)
    
    # add drivers for team
    tmp = results.join(drivers.drop(labels="number", axis=1), on='driverId')
    df = df.join(tmp.groupby(index).Driver.unique().str.join(', ').to_frame('Drivers'))

    use = ['Position', 'name', 'points', 'wins', 'podiums', 'scores', 'nationality', 'Drivers' ]
    df = df[use].set_index('Position')
    df.columns = df.columns.str.capitalize()
    return df

# Race results table
def format_results(df):
    df['Team'] = df.constructorId.map(constructors.name)
    df['Position'] = df.positionOrder
    use = ['Driver', 'Team', 'grid', 'Position', 'points', 'laps', 'time', 'status' ]
    df = df[use].sort_values('Position')
    df = df.set_index('Position')
    df.columns = df.columns.str.capitalize()
    return df

# Return the HTML img tag for a plot - allows us to set an alt tag for the image
# Added for accessibility and to fix warning:
# [NbConvertApp] WARNING | Alternative text is missing on 15 image(s).
def render_plot(title, alt_txt):
    fig = plt.gcf()
    fig.set_facecolor('white')
    buf = io.BytesIO()
    metadata = {'Title': title,
                'Author': 'James Trotman',
                'Source': f'https://www.kaggle.com/code/jtrotman/f1-race-traces-{YEAR}'}
    fig.savefig(buf, format='png', bbox_inches='tight', metadata=metadata)
    plt.close()
    b64 = base64.b64encode(buf.getvalue()).decode()
    return '<img alt="%s" src="data:image/png;base64,%s">' % (alt_txt, b64)

FileNotFoundError: [Errno 2] No such file or directory: '../input/formula-1-race-data/circuits.csv'

In [ ]:
plt.rc("figure", figsize=(16, 12))
plt.rc("font", size=(14))
plt.rc("axes", xmargin=0.01)

# Championship position traces
champ = driverStandings.groupby("driverId").position.last().to_frame("Pos")
champ = champ.join(drivers)
order = np.argsort(champ.Pos)

color = [DRIVER_C[d] for d in champ.index[order]]
style = [LINESTYLES[DRIVER_LS[d]] for d in champ.index[order]]
labels = champ.Pos.apply(formatter) + ". " + champ.display

chart = driverStandings.pivot(index="raceKey", columns="driverId", values="points")
names = races.set_index("raceKey").reindex(chart.index).name
names = names.str.replace("Grand Prix", "GP").rename("Race")
chart.index = names
chart.columns = labels

# Add origin
row = chart.iloc[0]
chart = pd.concat(((row * 0).to_frame("").T, chart))

chart.iloc[:, order].plot(title=f"F1 Drivers\' World Championship — {YEAR}", color=color, style=style)
plt.xticks(range(chart.shape[0]), chart.index, rotation=45)
plt.grid(axis="x", linestyle="--")
plt.ylabel("Points")
legend_opts = dict(bbox_to_anchor=(1.02, 0, 0.2, 1),
                   loc="upper right",
                   ncol=1,
                   shadow=True,
                   edgecolor="black",
                   mode="expand",
                   borderaxespad=0.)
plt.legend(**legend_opts)

html_lines = [
    f'<h2 id="drivers">Formula One Drivers\' World Championship &mdash; {YEAR}</h2>',
    render_plot(f'Formula One Drivers\' World Championship - {YEAR}',
                f'Formula One Drivers\' World Championship - {YEAR}'),
    table_html(drivers_standings(driverStandings.loc[driverStandings.index.max()]), "Results")
]
HTML("\n".join(html_lines))

In [ ]:
# Championship position traces
champ = constructorStandings.groupby("constructorId").position.last().to_frame("Pos")
champ = champ.join(constructors)
order = np.argsort(champ.Pos)

color = [TEAM_C[c] for c in champ.index[order]]
labels = champ.Pos.apply(formatter) + ". " + champ.label

chart = constructorStandings.pivot(index="raceKey", columns="constructorId", values="points")
names = races.set_index("raceKey").reindex(chart.index).name
names = names.str.replace("Grand Prix", "GP").rename("Race")
chart.index = names
chart.columns = labels

# Add origin
row = chart.iloc[0]
chart = pd.concat(((row * 0).to_frame("").T, chart))

chart.iloc[:, order].plot(title=f"F1 Constructors\' World Championship — {YEAR}", color=color)
plt.xticks(range(chart.shape[0]), chart.index, rotation=45)
plt.grid(axis="x", linestyle="--")
plt.ylabel("Points")
plt.legend(**legend_opts)

html_lines = [
    f'<h2 id="drivers">Formula One Constructors\' World Championship &mdash; {YEAR}</h2>',
    render_plot(f'Formula One Constructors\' World Championship - {YEAR}',
                f'Formula One Constructors\' World Championship - {YEAR}'),
    table_html(constructors_standings(constructorStandings.loc[constructorStandings.index.max()]), "Results")
]
HTML("\n".join(html_lines))

In [ ]:
# Show race traces
NEGATIVE_CUTOFF = -180
ANNOTATION_FONT_DICT = {"fontstyle":"italic", "fontsize":14}

def create_img_html(url, alt, style="display: inline-block;", width=16, height=16):
    return f'<img src="{url}" alt="{alt}" style="{style}" width="{width}" height="{height}">'

# Icons with their respective URLs
YT_IMG = create_img_html("https://youtube.com/favicon.ico", alt="YouTube icon")
WK_IMG = create_img_html("https://wikipedia.org/favicon.ico", alt="Wikipedia icon")
GM_IMG = create_img_html("https://maps.google.com/favicon.ico", alt="Google Maps icon")

driver_fastest_laps = Counter()
team_fastest_laps = Counter()

def header_html_lines(race):

    circuit = circuits.loc[race.circuitId]
    qstr = race["name"].replace(" ", "+")
    map_url = "https://www.google.com/maps/search/{lat}+{lng}".format(**circuit)
    vid_url = f"https://www.youtube.com/results?search_query=f1+{YEAR}+{qstr}"

    return [
        '<h2 id="race{round}">R{round} — {name}</h2>'.format(**race),
        f'[ <a href=#race{race["round"]-1}>&larr; prev</a> ]' if race['round'] > 1 else '',
        f'[ <a href=#race{race["round"]+1}>next &rarr;</a> ]' if race['round'] < len(races) else '',
        '<p><b>{date}</b> — '.format(img=WK_IMG, **race),
        '<b>Circuit:</b> <a href="{url}">{name}</a>, {location}, {country}'.format(**circuit),
        '<br><a href="{url}">{img} Wikipedia race report</a>'.format(img=WK_IMG, **race),
        f'<br><a href="{map_url}">{GM_IMG} Map Search</a>',
        f'<br><a href="{vid_url}">{YT_IMG} YouTube Search</a>',
    ]


for race_key, times in lapTimes.groupby(lapTimes.raceId.map(races["raceKey"])):

    race = races.loc[races.raceKey==race_key].squeeze()
    fullname = str(race["year"]) + " " + race["name"]
    title = "Round {round} — F1 {name} — {year}".format(**race)
    
    res = results.loc[results.raceId==race.name].set_index("driverId")
    res = res.join(drivers.drop(labels="number", axis=1))

    # Lap time data: One row per lap, one column per driver, values are lap time in seconds
    chart = times.pivot_table(values="seconds", index="lap", columns="driverId")

    sc = safety_cars.loc[safety_cars.Race==fullname][["Deployed", "Retreated"]]
    sc[["Deployed", "Retreated"]] -= 1
    sc = sc.fillna(len(chart)).astype(int)
    vsc = virtual_safety_cars.get(fullname, [])
    flags = red_flags.loc[red_flags.Race==fullname][["Lap"]]
    flags = flags.astype(int)

    annotation = ""
    if len(sc):
        lst = ", ".join(sc.Deployed.astype(str) + "-" + sc.Retreated.astype(str))
        annotation += f" Safety Car Laps: [{lst}]"
    if len(vsc):
        annotation += f" Virtual Safety Car Laps: {vsc}"
    if len(flags):
        lst = ", ".join(flags.Lap.astype(str))
        annotation += f" Red Flag: [{lst}]"

    # Re-order columns by race finish position for the legend
    labels = res.loc[chart.columns].apply(lambda r: "{positionOrder:2.0f}. {display}".format(**r), axis=1)
    order = np.argsort(labels)
    show = chart.iloc[:, order]

    basis = chart.median(axis=1).cumsum() # reference laptime series
    frontier = chart.cumsum().min(axis=1) # running best cumulative time

    # A red flag stoppage will create very long lap-times.
    # If this is late in the race & the cars are on different laps
    # (i.e. some are 1,2 or 3 laps down) the median time may be low for all those laps.
    # This means the overall median does not adjust the cumulative times enough,
    # this code corrects for that... e.g. Monaco 2011
    if any((frontier-basis)>100):
        adjust = ((frontier-basis)>100) * (frontier-basis).max()
        basis = (chart.median(axis=1) + adjust.diff().fillna(0)).cumsum()
    
    # Subtract reference from cumulative lap times
    show = -show.cumsum().subtract(basis, axis=0)

    # Fix large outliers (again, due to red flags), e.g. Australia 2016
    show[show>=800] = np.nan

    # Pitstops
    stops = pitStops.loc[pitStops.raceId==race.name].copy()
    if len(stops):
        # Brazil 2014 has pitstop times for Kevin Magnussen but no laptimes!
        stops = stops[stops.driverId.isin(chart.columns)]
        # Find x,y points for pitstops
        # (pd.DataFrame.lookup could do this with 1 line but it's deprecated)
        col_ix = list(map(show.columns.get_loc, stops.driverId))
        row_ix = list(map(show.index.get_loc, stops.lap))
        stops_y = show.to_numpy()[row_ix, col_ix]

    fastest_lap = times.iloc[np.argmin(np.asarray(times.seconds))]
    fastest_lap_y = show.loc[fastest_lap.lap, fastest_lap.driverId]

    # Lookup team color for this race (e.g. Russell 2020 Sakhir = Mercedes)
    driver_colors = res.constructorId.map(TEAM_C).to_dict()
    color = [driver_colors[d] for d in show.columns]
    style = [LINESTYLES[DRIVER_LS[d]] for d in show.columns]
    show.columns = labels.values[order]

    # Main Plot
    show.plot(title=title, style=style, color=color)
    plt.scatter(fastest_lap.lap,
                fastest_lap_y,
                s=200,
                marker='*',
                c=driver_colors[fastest_lap.driverId],
                alpha=.5)

    if len(stops):
        plt.scatter(stops.lap,
                    stops_y,
                    s=20,
                    marker='o',
                    c=list(map(driver_colors.get, stops.driverId)),
                    alpha=.5)

    top = show.max(axis=1).max()
    bottom = max(NEGATIVE_CUTOFF, show.min(axis=1).min())
    span = (top-bottom)
    ymin = bottom
    ymax = top+(span/50)

    # Add the shadow of where the lead car is compared to previous lap
    leader_line = add_lap_0(show).max(axis=1)
    leader_times = add_lap_0(chart).cumsum().min(axis=1).diff().shift(-1)
    plt.fill_between(leader_times.index,
                     (leader_line-leader_times).clip(NEGATIVE_CUTOFF),
                     -1000,
                     color='k',
                     alpha=.1)
    
    # Highlight safety cars
    for idx, row in sc.iterrows():
        plt.axvspan(row.Deployed, row.Retreated, color='#ffff00', alpha=.2);

    # Highlight virtual safety cars
    for lap in vsc:
        plt.axvspan(lap, lap+1, fc='#ff9933', ec=None, alpha=.2);

    # Highlight red flags
    for idx, row in flags.iterrows():
        plt.vlines(row.Lap, ymin, ymax, color='r', ls=':')

    # Finishing touches
    xticks = np.arange(0, len(chart)+1, 2)
    if len(chart) % 2:  # odd number of laps: nudge last tick to show it
        xticks[-1] += 1

    plt.xlabel("Lap", loc="right")
    plt.ylabel("Time Offset from Average Pace (s)")
    plt.xticks(xticks, xticks)
    plt.ylim(ymin, ymax)
    plt.grid(linestyle="--")
    plt.annotate(annotation, (10, -50), xycoords="axes pixels", **ANNOTATION_FONT_DICT);
    plt.legend(bbox_to_anchor=(0, -0.2, 1, 1),
               loc=(0, 0),
               ncol=6,
               shadow=True,
               edgecolor="black",
               mode="expand",
               borderaxespad=0.)

    driver_fastest_laps[fastest_lap.driverId.squeeze()] += 1
    team_fastest_laps[res.loc[fastest_lap.driverId].constructorId] += 1
    fastest_lap = fastest_lap.to_frame('').T.join(drivers, on='driverId')
    fastest_lap.columns = fastest_lap.columns.str.capitalize()

    html_lines = header_html_lines(race) + [
        render_plot('F1 {year} Round {round} — {name}'.format(**race),
                    'F1 {year} Round {round} — {name}'.format(**race)),
        table_html(fastest_lap[['Lap', 'Position', 'Time', 'Driver']], "Fastest Lap"),
        table_html(format_results(res), "Results")
    ]
    display(HTML("\n".join(html_lines)))

## Laps Per Position

In [ ]:
drivers_champ_df = driverStandings.groupby("driverId").position.last().to_frame("Pos")
drivers_champ_df = drivers_champ_df.sort_values("Pos").join(drivers)
labels = drivers_champ_df.Driver + " (" + drivers_champ_df.Pos.apply(formatter) + ")"
grid_df = lapTimes.groupby(["driverId", "position"]).size().unstack()
grid_df = grid_df.reindex(drivers_champ_df.index)
grid_df = grid_df.fillna(0).astype(int)
grid_df.index = grid_df.index.map(labels)
grid_df.style.background_gradient(axis=1, cmap="Blues")

## Top F1 Drivers 2021

In [ ]:
finished_status = status[status.status.str.match('^(Finished|\+\d+ Laps?)$')].index
results["win"] = (results["position"] == 1)
results["pole"] = (results["grid"] == 1)
results["top10"] = (results["grid"] <= 10)
results["finished"] = results.statusId.isin(finished_status)
results["dnf"] = ~results.statusId.isin(finished_status)


def make_summary(key, fastest_laps_dict):
    gb = results.groupby(key)

    summary_df = pd.DataFrame({
        "Laps": gb.laps.sum(),
        "Points": gb.points.sum(),
        "Wins": gb.win.sum(),
        "Podiums": gb.podium.sum(),
        "Scores": gb.score.sum(),
        "Poles": gb.pole.sum(),
        "Top 10 Starts": gb.top10.sum(),
        "Fastest Laps": fastest_laps_dict,
        "Finishes": gb.finished.sum(),
        "DNFs": gb.dnf.sum()
    })

    if 'sprint_results' in globals():
        # From 2022 onwards sprint race points are separate,
        # and are *not* included in points for main race
        sprint_points = sprint_results.groupby(key).points.sum()
        if sprint_points.count():
            summary_df["Points"] += sprint_points

    return summary_df.fillna(0)


def plot_summary(summary_df):
    facecolor = "white"
    fig = plt.figure(figsize=(18, 1 + len(summary_df) * .6), facecolor=facecolor)
    colors = plt.get_cmap("tab20").colors

    for i, col in enumerate(summary_df, start=1):
        ax = fig.add_subplot(1, summary_df.shape[1], i, facecolor=facecolor)
        edgecolors = np.where(summary_df[col] > 0, '#aaa', facecolor)
        ax.barh(summary_df.index, summary_df[col], color=colors[i], linewidth=1, edgecolor=edgecolors)
        ax.set_title(col, fontsize=16)
        ax.tick_params(left=False, bottom=False, right=False, top=False, labelleft=(i<=1))
        ax.xaxis.set_ticks([])

        for _, spine in ax.spines.items():
            spine.set_visible(False)

        for ind, val in summary_df[col].items():
            ax.text(0, ind, formatter(val), fontweight="bold")


driver_summary_df = make_summary('driverId', driver_fastest_laps)
labels = drivers_champ_df.display + " (" + drivers_champ_df.Pos.apply(formatter) + ")"
driver_summary_df = driver_summary_df.reindex(drivers_champ_df.head(10).index)[::-1]
driver_summary_df.index = driver_summary_df.index.map(labels)

In [ ]:
plot_summary(driver_summary_df)

plt.suptitle(f"Top F1 Drivers {YEAR}", fontsize=22, x=.12, y=.97, fontweight="bold");
plt.tight_layout()
# Save as svg and display, to avoid this graphic becoming the notebook thumbnail image
buf = io.BytesIO()
plt.savefig(buf, bbox_inches='tight', format='svg')
plt.close()
HTML(buf.getvalue().decode('utf-8'))

## F1 Teams 2021

In [ ]:
teams_champ_df = constructorStandings.groupby("constructorId").position.last().to_frame("Pos")
teams_champ_df = teams_champ_df.sort_values("Pos").join(constructors)

team_summary_df = make_summary('constructorId', team_fastest_laps)
labels = teams_champ_df.label + " (" + teams_champ_df.Pos.apply(formatter) + ")"
team_summary_df = team_summary_df.reindex(teams_champ_df.index)[::-1]
team_summary_df.index = team_summary_df.index.map(labels)

plot_summary(team_summary_df)

plt.suptitle(f"F1 Teams {YEAR}", fontsize=22, x=.12, y=.97, fontweight="bold");
plt.tight_layout()
# Save as svg and display, to avoid this graphic becoming the notebook thumbnail image
buf = io.BytesIO()
plt.savefig(buf, bbox_inches='tight', format='svg')
plt.close()
HTML(buf.getvalue().decode('utf-8'))

In [ ]:
HTML("""<script>fill_toc()</script>""")

## More F1 Race Traces

[1996](https://www.kaggle.com/code/jtrotman/f1-race-traces-1996), 
[1997](https://www.kaggle.com/code/jtrotman/f1-race-traces-1997), 
[1998](https://www.kaggle.com/code/jtrotman/f1-race-traces-1998), 
[1999](https://www.kaggle.com/code/jtrotman/f1-race-traces-1999), 
[2000](https://www.kaggle.com/code/jtrotman/f1-race-traces-2000), 
[2001](https://www.kaggle.com/code/jtrotman/f1-race-traces-2001), 
[2002](https://www.kaggle.com/code/jtrotman/f1-race-traces-2002), 
[2003](https://www.kaggle.com/code/jtrotman/f1-race-traces-2003), 
[2004](https://www.kaggle.com/code/jtrotman/f1-race-traces-2004), 
[2005](https://www.kaggle.com/code/jtrotman/f1-race-traces-2005), 
[2006](https://www.kaggle.com/code/jtrotman/f1-race-traces-2006), 
[2007](https://www.kaggle.com/code/jtrotman/f1-race-traces-2007), 
[2008](https://www.kaggle.com/code/jtrotman/f1-race-traces-2008), 
[2009](https://www.kaggle.com/code/jtrotman/f1-race-traces-2009), 
[2010](https://www.kaggle.com/code/jtrotman/f1-race-traces-2010), 
[2011](https://www.kaggle.com/code/jtrotman/f1-race-traces-2011), 
[2012](https://www.kaggle.com/code/jtrotman/f1-race-traces-2012), 
[2013](https://www.kaggle.com/code/jtrotman/f1-race-traces-2013), 
[2014](https://www.kaggle.com/code/jtrotman/f1-race-traces-2014), 
[2015](https://www.kaggle.com/code/jtrotman/f1-race-traces-2015), 
[2016](https://www.kaggle.com/code/jtrotman/f1-race-traces-2016), 
[2017](https://www.kaggle.com/code/jtrotman/f1-race-traces-2017), 
[2018](https://www.kaggle.com/code/jtrotman/f1-race-traces-2018), 
[2019](https://www.kaggle.com/code/jtrotman/f1-race-traces-2019), 
[2020](https://www.kaggle.com/code/jtrotman/f1-race-traces-2020), 
[2021](https://www.kaggle.com/code/jtrotman/f1-race-traces-2021), 
[2022](https://www.kaggle.com/code/jtrotman/f1-race-traces-2022), 
[2023](https://www.kaggle.com/code/jtrotman/f1-race-traces-2023), 
[2024](https://www.kaggle.com/code/jtrotman/f1-race-traces-2024), 
[2025](https://www.kaggle.com/code/jtrotman/f1-race-traces-2025).


## See Also

This [notebook shows the same idea for one MotoGP race](https://www.kaggle.com/jtrotman/motogp-race-traces-from-pdf), and explores several ways of adjusting the plots to highlight new details.


*This notebook uses material from the Wikipedia article <a href="https://en.wikipedia.org/wiki/2021_Formula_One_World_Championship">"2021 Formula One World Championship"</a>, which is released under the <a href="https://creativecommons.org/licenses/by-sa/3.0/">Creative Commons Attribution-Share-Alike License 3.0</a>.*